In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 212.3MB 59kB/s 
     |████████████████████████████████| 204kB 36.7MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=06dab51df077eeb64b79c2cb9a8b9a3e7b8e7eff00034d62d68a992b392e6481
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [3]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

## 1- Se tiene un RDD con el registro de notas de los alumnos de la forma (padrón, materia, nota, fecha). Se pide resolver utilizando PySpark:


In [5]:
registroDeNotas = [(100946, 'Algebra II', 10, '2019-08-15'),
 (100940, 'Fisica III', 10, '2019-10-27'),
 (100965, 'Quimica', 8, '2017-08-31'),
 (100958, 'Taller 1', 4, '2018-12-28'),
 (100975, 'Fisica III', 3, '2017-08-11'),
 (100930, 'Taller 1', 4, '2017-04-24'),
 (100946, 'Laboratorio', 2, '2019-01-18'),
 (100991, 'Algebra II', 1, '2018-03-29'),
 (100966, 'Laboratorio', 6, '2018-08-20'),
 (100985, 'Quimica', 9, '2019-03-28'),
 (100918, 'Algebra II', 5, '2018-09-14'),
 (101000, 'Fisica III', 4, '2017-12-13'),
 (100937, 'Quimica', 7, '2019-07-15'),
 (100903, 'Laboratorio', 3, '2018-09-06'),
 (100941, 'Fisica III', 6, '2018-06-23'),
 (100962, 'Taller 1', 3, '2019-06-13'),
 (100938, 'Fisica III', 7, '2017-10-04'),
 (100986, 'Algebra II', 3, '2017-05-06'),
 (100902, 'Quimica', 3, '2017-08-15'),
 (100950, 'Taller 1', 7, '2017-08-20'),
 (100911, 'Laboratorio', 7, '2018-02-03'),
 (101000, 'Fisica III', 1, '2018-07-10'),
 (100968, 'Algebra II', 4, '2019-04-10'),
 (100951, 'Quimica', 3, '2018-11-04'),
 (100944, 'Taller 1', 10, '2017-04-01'),
 (100912, 'Quimica', 10, '2019-05-19'),
 (100993, 'Taller 1', 2, '2019-05-05'),
 (100934, 'Algebra II', 8, '2017-12-16'),
 (100966, 'Taller 1', 1, '2019-08-06'),
 (100934, 'Quimica', 3, '2017-12-25'),
 (100968, 'Laboratorio', 10, '2019-11-12'),
 (100940, 'Algebra II', 2, '2017-11-29'),
 (100986, 'Algebra II', 9, '2019-03-05'),
 (100977, 'Laboratorio', 7, '2019-10-24'),
 (100926, 'Quimica', 9, '2018-03-16'),
 (100919, 'Taller 1', 7, '2019-06-05'),
 (100930, 'Fisica III', 9, '2018-05-17'),
 (100990, 'Fisica III', 8, '2018-03-25'),
 (100951, 'Taller 1', 6, '2017-10-07'),
 (100907, 'Algebra II', 3, '2017-08-01'),
 (100937, 'Quimica', 8, '2018-06-01'),
 (100928, 'Taller 1', 7, '2018-07-16'),
 (100956, 'Taller 1', 8, '2018-06-08'),
 (100928, 'Laboratorio', 2, '2018-05-15'),
 (100945, 'Fisica III', 5, '2017-04-11'),
 (100981, 'Laboratorio', 7, '2019-03-26'),
 (100970, 'Algebra II', 9, '2019-11-04'),
 (100910, 'Laboratorio', 1, '2017-11-12'),
 (100960, 'Taller 1', 1, '2017-04-18'),
 (100985, 'Taller 1', 6, '2017-12-25')]
len(registroDeNotas)

50

### A) Cuántos alumnos aprobaron al menos 1 materia en los últimos 2 años.

In [6]:
registroDeNotasRDD = sc.parallelize(registroDeNotas, 8)
registroDeNotasRDD.take(5)

[(100946, 'Algebra II', 10, '2019-08-15'),
 (100940, 'Fisica III', 10, '2019-10-27'),
 (100965, 'Quimica', 8, '2017-08-31'),
 (100958, 'Taller 1', 4, '2018-12-28'),
 (100975, 'Fisica III', 3, '2017-08-11')]

In [7]:
FechasFiltradas = registroDeNotasRDD.filter(lambda x: x[3] >= '2018-06-06')
FechasFiltradas.take(10)

[(100946, 'Algebra II', 10, '2019-08-15'),
 (100940, 'Fisica III', 10, '2019-10-27'),
 (100958, 'Taller 1', 4, '2018-12-28'),
 (100946, 'Laboratorio', 2, '2019-01-18'),
 (100966, 'Laboratorio', 6, '2018-08-20'),
 (100985, 'Quimica', 9, '2019-03-28'),
 (100918, 'Algebra II', 5, '2018-09-14'),
 (100937, 'Quimica', 7, '2019-07-15'),
 (100903, 'Laboratorio', 3, '2018-09-06'),
 (100941, 'Fisica III', 6, '2018-06-23')]

In [8]:
FechasNotasFiltradas = FechasFiltradas.filter(lambda x: x[2] >= 4 )
FechasNotasFiltradas.take(10)

[(100946, 'Algebra II', 10, '2019-08-15'),
 (100940, 'Fisica III', 10, '2019-10-27'),
 (100958, 'Taller 1', 4, '2018-12-28'),
 (100966, 'Laboratorio', 6, '2018-08-20'),
 (100985, 'Quimica', 9, '2019-03-28'),
 (100918, 'Algebra II', 5, '2018-09-14'),
 (100937, 'Quimica', 7, '2019-07-15'),
 (100941, 'Fisica III', 6, '2018-06-23'),
 (100968, 'Algebra II', 4, '2019-04-10'),
 (100912, 'Quimica', 10, '2019-05-19')]

Lo puedo hacer con 2 filter o en un solo

In [9]:
prueba = registroDeNotasRDD.filter(lambda x: x[3] >= '2018-06-06' and x[2] >= 4 )
prueba.take(10)

[(100946, 'Algebra II', 10, '2019-08-15'),
 (100940, 'Fisica III', 10, '2019-10-27'),
 (100958, 'Taller 1', 4, '2018-12-28'),
 (100966, 'Laboratorio', 6, '2018-08-20'),
 (100985, 'Quimica', 9, '2019-03-28'),
 (100918, 'Algebra II', 5, '2018-09-14'),
 (100937, 'Quimica', 7, '2019-07-15'),
 (100941, 'Fisica III', 6, '2018-06-23'),
 (100968, 'Algebra II', 4, '2019-04-10'),
 (100912, 'Quimica', 10, '2019-05-19')]

In [10]:
#Lo converti en un clave valor para aplicar reducedbykey
final = prueba.map(lambda x: (x[0], x[2])).cache()
final.take(10)

[(100946, 10),
 (100940, 10),
 (100958, 4),
 (100966, 6),
 (100985, 9),
 (100918, 5),
 (100937, 7),
 (100941, 6),
 (100968, 4),
 (100912, 10)]

In [11]:
resultado = final.reduceByKey(lambda a,b: a+b)
resultado.count()

17

### B) Un RDD conteniendo el promedio de notas de cada alumno de la forma (padrón, promedio).

In [12]:
FiltroPadronesNotas = registroDeNotasRDD.map(lambda x: (x[0], (x[2],1)))
FiltroPadronesNotas.take(5)

[(100946, (10, 1)),
 (100940, (10, 1)),
 (100965, (8, 1)),
 (100958, (4, 1)),
 (100975, (3, 1))]

In [13]:
Agrupado = FiltroPadronesNotas.reduceByKey(lambda a,b: (a[0]+b[0],a[1]+b[1]))
Agrupado.take(5)

[(101000, (5, 2)),
 (100968, (14, 2)),
 (100944, (10, 1)),
 (100912, (10, 1)),
 (100928, (9, 2))]

In [14]:
Final = Agrupado.map(lambda x: (x[0],(x[1][0]/x[1][1]))).cache()
Final.take(5)

[(101000, 2.5), (100968, 7.0), (100944, 10.0), (100912, 10.0), (100928, 4.5)]

### C) El nombre y apellido del alumno con mejor promedio. Para esto puede utilizarse un segundo RDD alumnos con registros (padron, nombre y apellido).

In [15]:
Prueba = Final.reduce(lambda a,b: a if a[1]>=b[1] else b)
Prueba

(100944, 10.0)

In [16]:
ParteC = Final.takeOrdered(5, key=lambda x: -x[1] )
ParteC

[(100944, 10.0), (100912, 10.0), (100970, 9.0), (100926, 9.0), (100956, 8.0)]

## 2- Se tiene un RDD registros de ventas de producto con la forma (fecha de venta, código de producto, precio de venta) y en otro RDD detalle de los productos con (código de producto, descripción del producto, categoría). Se pide resolver utilizando PySpark:

In [17]:
Ventas = [('2018-08-17', 9, 6944.21),
 ('2018-01-05', 5, 5231.64),
 ('2018-08-26', 6, 10762.98),
 ('2018-12-31', 9, 4940.65),
 ('2018-05-26', 9, 8881.46),
 ('2019-12-03', 4, 17657.29),
 ('2018-01-12', 7, 7813.6),
 ('2019-04-03', 6, 589.28),
 ('2018-12-22', 5, 10354.49),
 ('2018-10-18', 7, 18793.87),
 ('2019-01-01', 2, 8056.1),
 ('2019-04-03', 10, 18322.44),
 ('2018-02-08', 6, 16532.41),
 ('2018-01-13', 6, 3197.6),
 ('2018-07-01', 1, 1871.5),
 ('2018-02-01', 4, 15477.1),
 ('2019-04-01', 3, 15411.98),
 ('2019-03-29', 4, 17192.59),
 ('2019-09-06', 2, 5457.19),
 ('2018-11-29', 8, 1440.88),
 ('2018-10-03', 5, 18664.71),
 ('2019-06-03', 8, 16725.33),
 ('2019-07-12', 4, 10859.8),
 ('2019-04-09', 1, 6547.69),
 ('2018-08-13', 6, 9512.12),
 ('2018-06-11', 8, 4462.42),
 ('2018-08-15', 3, 19025.53),
 ('2018-02-10', 10, 19846.64),
 ('2018-11-13', 9, 17315.01),
 ('2019-06-05', 3, 5768.77),
 ('2019-04-12', 5, 9315.18),
 ('2018-07-20', 2, 5810.28),
 ('2019-10-13', 1, 15204.76),
 ('2018-01-09', 4, 1320.14),
 ('2019-10-25', 4, 4385.27),
 ('2018-06-25', 1, 1923.72),
 ('2018-04-24', 9, 2179.16),
 ('2018-05-28', 2, 14439.38),
 ('2019-05-06', 1, 12467.74),
 ('2019-05-15', 2, 9131.1),
 ('2018-08-11', 4, 6662.29),
 ('2018-11-28', 9, 12818.72),
 ('2019-09-04', 1, 3957.78),
 ('2019-10-27', 3, 4764.5),
 ('2018-03-06', 10, 10442.48),
 ('2019-10-27', 8, 17393.2),
 ('2018-05-27', 7, 19371.23),
 ('2019-04-23', 10, 5497.02),
 ('2019-05-23', 2, 9813.6),
 ('2018-04-04', 1, 6894.07),
 ('2018-11-06', 7, 12218.35),
 ('2018-11-17', 9, 7412.2),
 ('2019-05-25', 7, 13134.33),
 ('2018-01-18', 2, 16991.42),
 ('2019-03-22', 10, 10298.14),
 ('2018-09-22', 10, 18334.04),
 ('2019-06-09', 2, 4398.7),
 ('2018-02-07', 1, 13944.03),
 ('2018-04-27', 4, 12617.54),
 ('2018-09-15', 1, 15048.38),
 ('2018-12-15', 1, 9763.23),
 ('2019-04-13', 10, 2579.66),
 ('2018-02-15', 1, 5081.16),
 ('2018-09-10', 9, 18280.73),
 ('2019-07-28', 7, 10093.87),
 ('2019-10-31', 2, 19158.44),
 ('2018-06-14', 4, 3430.72),
 ('2019-09-06', 6, 889.75),
 ('2018-10-04', 4, 363.73),
 ('2019-08-28', 3, 11082.9),
 ('2019-03-28', 3, 442.17),
 ('2019-02-07', 6, 1743.11),
 ('2019-03-19', 10, 9298.12),
 ('2018-04-23', 7, 4970.97),
 ('2019-01-15', 4, 8242.85),
 ('2018-02-22', 4, 14056.74),
 ('2019-02-08', 1, 19416.55),
 ('2019-06-17', 5, 11060.26),
 ('2018-06-03', 4, 11565.88),
 ('2019-09-13', 5, 10586.13),
 ('2018-12-12', 6, 995.86),
 ('2019-05-12', 2, 16003.65),
 ('2019-04-04', 9, 13854.83),
 ('2018-09-18', 4, 11082.09),
 ('2019-07-18', 9, 17570.57),
 ('2018-08-10', 1, 4500.82),
 ('2018-04-12', 6, 16689.03),
 ('2019-10-22', 6, 17413.9),
 ('2019-01-19', 8, 8413.12),
 ('2018-02-04', 6, 11624.39),
 ('2018-05-15', 9, 11767.33),
 ('2019-02-25', 7, 2117.35),
 ('2018-01-03', 4, 2211.06),
 ('2018-02-21', 10, 11167.33),
 ('2018-09-15', 4, 3265.94),
 ('2018-08-31', 8, 19424.2),
 ('2019-12-21', 5, 402.44),
 ('2019-06-10', 2, 1556.82),
 ('2018-10-10', 7, 3576.89),
 ('2018-05-20', 2, 18966.65),
 ('2019-01-26', 10, 17926.15),
 ('2018-09-13', 9, 2105.0),
 ('2019-09-03', 6, 2269.98),
 ('2019-05-02', 8, 17130.52),
 ('2019-11-30', 4, 16253.21),
 ('2018-02-14', 8, 6102.35),
 ('2018-01-10', 4, 6841.23),
 ('2019-04-02', 4, 15139.42),
 ('2019-05-01', 8, 7328.78),
 ('2018-11-16', 7, 15221.63),
 ('2018-12-19', 6, 12774.09),
 ('2019-07-04', 8, 5960.46),
 ('2019-02-20', 3, 111.04),
 ('2018-08-19', 2, 19061.43),
 ('2018-06-29', 10, 408.53),
 ('2018-04-05', 1, 6480.78),
 ('2018-04-12', 6, 2275.01),
 ('2019-11-01', 7, 5367.44),
 ('2018-05-20', 10, 414.42),
 ('2018-09-24', 7, 10096.47),
 ('2019-08-06', 2, 8782.32),
 ('2019-02-24', 4, 12026.74),
 ('2018-09-26', 1, 6024.14),
 ('2019-05-04', 2, 14500.65),
 ('2018-10-08', 5, 10014.94),
 ('2019-10-05', 5, 1291.18),
 ('2018-06-16', 2, 14488.52),
 ('2018-01-09', 1, 5808.21),
 ('2019-06-03', 1, 9869.14),
 ('2018-04-13', 7, 6548.52),
 ('2019-07-08', 3, 19880.2),
 ('2018-11-04', 4, 4437.12),
 ('2018-11-02', 5, 16538.64),
 ('2019-05-11', 9, 10868.06),
 ('2019-07-10', 5, 1947.68),
 ('2018-06-15', 4, 17120.61),
 ('2019-08-19', 7, 14402.7),
 ('2018-11-24', 10, 1834.21),
 ('2018-03-31', 2, 8674.65),
 ('2018-08-09', 6, 19758.83),
 ('2018-01-09', 3, 16329.02),
 ('2018-07-16', 1, 5205.69),
 ('2019-09-28', 6, 6196.5),
 ('2018-03-20', 7, 1119.74),
 ('2018-03-31', 5, 17194.21),
 ('2018-01-23', 7, 7008.08),
 ('2018-04-19', 3, 2765.56),
 ('2018-02-20', 1, 3866.09),
 ('2019-01-02', 2, 9712.87),
 ('2018-08-07', 6, 15676.93),
 ('2018-02-25', 1, 3368.34),
 ('2019-07-09', 8, 8194.44),
 ('2019-09-08', 8, 11634.75),
 ('2019-07-02', 3, 8753.69),
 ('2019-12-26', 7, 1444.62),
 ('2018-10-14', 7, 8485.71),
 ('2018-07-14', 10, 4140.54),
 ('2019-12-05', 2, 15856.18),
 ('2018-08-21', 9, 7181.51),
 ('2019-03-31', 8, 7489.12),
 ('2019-05-17', 6, 3233.77),
 ('2018-08-06', 5, 13416.57),
 ('2018-11-03', 5, 12704.33),
 ('2018-01-02', 9, 18647.31),
 ('2019-05-03', 3, 6670.33),
 ('2019-05-27', 4, 19557.04),
 ('2019-06-03', 5, 12322.11),
 ('2018-04-30', 5, 4625.85),
 ('2019-03-02', 2, 13132.1),
 ('2019-03-07', 6, 14929.98),
 ('2019-12-01', 3, 16095.21),
 ('2019-06-15', 6, 12285.91),
 ('2018-12-17', 3, 3800.22),
 ('2019-10-15', 6, 5102.53),
 ('2018-12-26', 1, 15432.84),
 ('2018-12-15', 1, 2110.76),
 ('2018-04-20', 10, 12976.15),
 ('2019-05-13', 3, 12749.2),
 ('2018-03-16', 3, 227.94),
 ('2019-12-05', 7, 18206.0),
 ('2018-03-31', 2, 3914.28),
 ('2019-04-16', 5, 693.49),
 ('2018-11-15', 8, 16626.92),
 ('2018-09-14', 4, 787.39),
 ('2019-07-06', 5, 15554.96),
 ('2019-01-24', 8, 17435.82),
 ('2019-04-06', 2, 7386.25),
 ('2018-05-12', 10, 14514.81),
 ('2019-06-17', 3, 2971.9),
 ('2018-01-19', 7, 14751.16),
 ('2018-03-24', 6, 14638.92),
 ('2019-02-07', 8, 13630.68),
 ('2019-06-16', 1, 5322.71),
 ('2019-10-17', 10, 12939.0),
 ('2019-10-16', 7, 6174.63),
 ('2019-08-07', 10, 213.97),
 ('2019-02-16', 3, 11281.63),
 ('2019-10-22', 10, 3889.9),
 ('2019-12-01', 2, 105.97),
 ('2019-09-11', 1, 4211.18),
 ('2019-01-25', 9, 3871.41),
 ('2018-08-17', 7, 2382.34),
 ('2018-09-12', 1, 896.34),
 ('2018-12-31', 1, 10121.75),
 ('2018-10-16', 5, 5762.24),
 ('2019-06-03', 5, 9549.21),
 ('2018-06-05', 9, 16956.06),
 ('2018-06-27', 7, 93.3),
 ('2019-05-08', 9, 18588.47),
 ('2018-10-18', 1, 4285.78),
 ('2019-11-01', 1, 7644.12),
 ('2019-09-28', 4, 412.68),
 ('2018-10-15', 6, 17375.17),
 ('2019-09-11', 8, 5497.63),
 ('2018-12-31', 5, 10019.92),
 ('2018-10-17', 7, 276.01),
 ('2018-07-30', 6, 140.07),
 ('2019-03-23', 5, 1333.67),
 ('2018-06-18', 7, 7386.52),
 ('2019-07-14', 10, 16110.71),
 ('2018-03-25', 5, 13490.96),
 ('2018-11-19', 9, 11509.19),
 ('2018-09-26', 4, 8826.16),
 ('2019-03-15', 9, 9105.46),
 ('2019-05-16', 9, 5797.55),
 ('2019-10-10', 1, 7560.45),
 ('2019-03-12', 1, 8929.79),
 ('2018-05-18', 7, 6291.34),
 ('2019-07-05', 7, 14301.24),
 ('2019-08-29', 9, 3675.43),
 ('2019-08-24', 7, 11706.06),
 ('2018-05-20', 9, 17123.36),
 ('2019-12-11', 4, 7978.24),
 ('2018-05-23', 1, 56.95),
 ('2019-05-11', 6, 10295.79),
 ('2018-12-10', 5, 13696.17),
 ('2018-09-11', 4, 18257.43),
 ('2019-12-21', 8, 9963.93),
 ('2018-11-26', 3, 17942.07),
 ('2018-03-15', 8, 13611.75),
 ('2018-02-27', 3, 18452.1),
 ('2018-10-16', 9, 4472.84),
 ('2018-06-26', 6, 6159.87),
 ('2019-10-07', 7, 15691.96),
 ('2018-04-15', 6, 2503.56),
 ('2018-10-29', 8, 7110.51),
 ('2018-10-23', 10, 12834.16),
 ('2018-02-28', 7, 1458.33),
 ('2018-04-10', 2, 457.13),
 ('2018-09-06', 8, 7529.88),
 ('2018-01-15', 5, 11960.33),
 ('2019-05-14', 9, 13706.54),
 ('2018-05-14', 7, 13214.66),
 ('2019-08-28', 8, 11386.79),
 ('2019-08-28', 5, 5021.76),
 ('2018-01-31', 5, 9971.71),
 ('2018-04-14', 8, 13044.52),
 ('2018-09-19', 7, 13804.56),
 ('2018-01-28', 7, 13568.9),
 ('2018-10-14', 9, 4467.89),
 ('2019-08-29', 9, 11892.84),
 ('2019-07-18', 6, 17967.07),
 ('2019-11-17', 10, 15965.92),
 ('2018-06-12', 7, 19543.6),
 ('2018-08-04', 5, 18123.42),
 ('2018-12-24', 6, 4507.15),
 ('2018-07-31', 10, 3488.64),
 ('2018-04-14', 9, 18384.73),
 ('2018-10-23', 7, 5806.66),
 ('2019-11-03', 5, 473.05),
 ('2019-02-26', 9, 15357.47),
 ('2018-09-13', 6, 10067.38),
 ('2019-04-11', 1, 18493.89),
 ('2019-07-22', 9, 13526.12),
 ('2018-06-05', 4, 16460.42),
 ('2018-10-25', 4, 17231.37),
 ('2018-08-19', 3, 3319.93),
 ('2018-01-24', 8, 10212.58),
 ('2019-03-06', 4, 15742.8),
 ('2018-11-09', 5, 17481.46),
 ('2019-12-11', 4, 19974.25),
 ('2019-12-01', 5, 14738.07),
 ('2019-11-28', 8, 19918.22),
 ('2019-09-05', 3, 1223.33),
 ('2018-07-20', 1, 8569.63),
 ('2018-11-13', 6, 3036.25),
 ('2018-08-23', 8, 8056.37),
 ('2018-10-20', 6, 14866.82),
 ('2019-06-18', 9, 6277.62),
 ('2019-06-20', 7, 10356.8),
 ('2018-12-13', 7, 2615.6),
 ('2018-01-16', 3, 6535.17),
 ('2018-01-09', 4, 11464.65),
 ('2018-08-06', 3, 9430.65),
 ('2018-07-11', 6, 8900.63),
 ('2018-02-16', 4, 361.54),
 ('2018-05-21', 2, 8168.23),
 ('2018-04-27', 3, 11166.54),
 ('2019-01-10', 10, 4059.99),
 ('2019-11-11', 9, 16345.91),
 ('2018-07-11', 9, 14906.52),
 ('2018-07-27', 2, 8273.75),
 ('2018-09-29', 5, 4586.16),
 ('2019-05-24', 9, 4753.19),
 ('2018-12-23', 1, 5620.8),
 ('2018-07-07', 9, 11520.74),
 ('2018-08-17', 5, 14995.15),
 ('2018-06-01', 1, 917.49),
 ('2018-11-19', 3, 18525.37),
 ('2019-09-28', 8, 6865.6),
 ('2019-10-19', 1, 11555.7),
 ('2019-03-12', 3, 16218.82),
 ('2019-10-02', 1, 2360.17),
 ('2018-03-18', 4, 11717.73),
 ('2018-02-16', 3, 13881.86),
 ('2019-02-04', 1, 2393.69),
 ('2019-01-26', 5, 12020.59),
 ('2019-06-29', 8, 1486.88),
 ('2018-09-03', 1, 2688.19),
 ('2019-07-30', 1, 5026.05),
 ('2019-07-29', 1, 9216.45),
 ('2019-11-02', 2, 5318.49),
 ('2018-07-10', 9, 10700.47),
 ('2018-01-01', 7, 7080.94),
 ('2019-04-03', 1, 18220.48),
 ('2019-05-01', 2, 1713.99),
 ('2019-01-21', 8, 19637.25),
 ('2018-10-16', 1, 5166.68),
 ('2019-11-11', 7, 17318.14),
 ('2018-01-19', 1, 19847.79),
 ('2019-06-27', 5, 605.66),
 ('2019-11-08', 4, 5824.28),
 ('2018-10-12', 4, 16616.79),
 ('2018-10-22', 10, 9660.25),
 ('2019-03-05', 2, 12424.66),
 ('2018-04-09', 6, 11068.97),
 ('2019-12-16', 10, 14338.66),
 ('2018-11-04', 6, 9752.07),
 ('2019-11-27', 5, 17467.15),
 ('2018-10-05', 9, 2610.65),
 ('2018-04-05', 2, 5393.15),
 ('2019-07-06', 5, 18966.43),
 ('2019-08-25', 4, 9574.83),
 ('2018-01-31', 3, 19580.2),
 ('2018-02-10', 2, 6701.97),
 ('2019-10-12', 3, 4754.38),
 ('2019-07-08', 9, 1091.59),
 ('2019-10-22', 1, 12621.44),
 ('2019-10-07', 3, 10488.36),
 ('2018-09-28', 8, 15902.75),
 ('2018-10-01', 1, 19299.38),
 ('2019-07-02', 9, 9571.76),
 ('2018-04-22', 1, 2388.25),
 ('2018-07-07', 10, 19656.47),
 ('2018-01-30', 2, 11461.31),
 ('2019-10-02', 7, 12850.94),
 ('2018-07-17', 10, 14715.54),
 ('2019-05-16', 5, 16641.24),
 ('2019-07-09', 8, 6926.49),
 ('2019-09-26', 9, 3811.98),
 ('2019-11-22', 6, 6401.6),
 ('2019-01-14', 7, 17401.09),
 ('2018-01-02', 2, 2054.46),
 ('2019-08-27', 6, 5293.82),
 ('2019-08-22', 7, 508.31),
 ('2019-12-23', 4, 10017.97),
 ('2019-05-04', 7, 12689.64),
 ('2019-08-22', 9, 8888.13),
 ('2019-11-22', 6, 4944.98),
 ('2019-06-13', 10, 7560.68),
 ('2018-03-24', 9, 8940.1),
 ('2018-10-01', 7, 18984.75),
 ('2018-10-27', 2, 639.32),
 ('2018-03-03', 3, 17032.09),
 ('2018-09-11', 2, 13454.67),
 ('2019-05-04', 7, 5346.66),
 ('2018-08-22', 3, 544.6),
 ('2019-02-25', 10, 12072.46),
 ('2019-01-01', 9, 17848.86),
 ('2019-04-26', 1, 9950.74),
 ('2019-08-24', 4, 15657.43),
 ('2019-04-01', 1, 19675.0),
 ('2018-02-21', 6, 17059.8),
 ('2018-02-11', 9, 13198.17),
 ('2019-01-05', 4, 9542.42),
 ('2018-10-01', 6, 12700.59),
 ('2019-09-30', 1, 8934.86),
 ('2019-01-07', 3, 8119.14),
 ('2019-08-08', 9, 16727.86),
 ('2018-10-02', 3, 1523.8),
 ('2019-01-24', 3, 6962.13),
 ('2019-09-30', 4, 5464.49),
 ('2018-11-12', 3, 11687.68),
 ('2018-03-17', 5, 2998.11),
 ('2019-04-15', 2, 11848.03),
 ('2018-10-03', 4, 14900.06),
 ('2018-09-29', 9, 12073.21),
 ('2018-02-06', 3, 7211.38),
 ('2018-05-15', 7, 7377.6),
 ('2019-11-25', 1, 19347.99),
 ('2019-06-28', 10, 2524.04),
 ('2018-11-21', 7, 11167.83),
 ('2018-06-28', 8, 6932.95),
 ('2019-05-16', 7, 12061.6),
 ('2019-07-20', 6, 12136.68),
 ('2018-11-16', 6, 7632.54),
 ('2019-11-26', 6, 10230.35),
 ('2019-07-03', 5, 13802.36),
 ('2019-12-15', 1, 10564.66),
 ('2019-02-08', 9, 2594.52),
 ('2018-02-01', 8, 16455.96),
 ('2019-05-25', 4, 13474.67),
 ('2018-05-31', 7, 3868.13),
 ('2019-01-26', 9, 14812.27),
 ('2019-07-27', 10, 253.47),
 ('2018-09-14', 3, 10147.48),
 ('2019-05-31', 2, 11234.9),
 ('2019-03-14', 4, 14287.39),
 ('2019-05-11', 4, 3498.33),
 ('2018-01-04', 8, 82.67),
 ('2018-11-11', 4, 5360.99),
 ('2018-05-06', 2, 2911.23),
 ('2018-04-12', 4, 16005.49),
 ('2018-08-20', 4, 6631.18),
 ('2018-12-25', 6, 8555.29),
 ('2019-04-11', 1, 13160.65),
 ('2019-03-01', 9, 5538.98),
 ('2019-07-26', 9, 19614.34),
 ('2018-09-22', 2, 19430.85),
 ('2018-03-23', 10, 13389.67),
 ('2019-12-16', 4, 11248.81),
 ('2019-09-14', 1, 4489.27),
 ('2018-07-16', 10, 273.98),
 ('2018-07-11', 3, 13165.93),
 ('2019-11-17', 2, 13988.99),
 ('2018-09-18', 8, 745.76),
 ('2018-07-15', 9, 7466.69),
 ('2018-04-01', 8, 10132.18),
 ('2018-07-11', 8, 1161.0),
 ('2018-05-04', 7, 6225.76),
 ('2019-08-08', 10, 4998.44),
 ('2019-08-19', 8, 19451.14),
 ('2019-11-25', 6, 10307.78),
 ('2019-12-21', 10, 3368.38),
 ('2018-10-21', 1, 2850.57),
 ('2019-05-07', 8, 8748.27),
 ('2019-01-06', 3, 10050.84),
 ('2018-11-10', 10, 11615.94),
 ('2019-04-16', 10, 2142.32),
 ('2019-08-21', 9, 19164.59),
 ('2019-09-10', 2, 9905.69),
 ('2018-10-15', 9, 6151.02),
 ('2019-12-07', 10, 16882.66),
 ('2019-07-08', 1, 19405.53),
 ('2018-12-08', 7, 7650.07),
 ('2019-10-19', 6, 19062.36),
 ('2018-10-23', 7, 9050.87),
 ('2018-06-14', 6, 5375.9),
 ('2018-10-25', 8, 7694.15),
 ('2019-04-04', 7, 14815.94),
 ('2018-11-03', 1, 12098.57),
 ('2018-08-04', 9, 13830.7),
 ('2019-06-05', 6, 13916.67),
 ('2018-05-29', 8, 13076.0),
 ('2018-11-26', 2, 14078.2),
 ('2019-06-11', 3, 19084.5),
 ('2018-05-02', 2, 6302.27),
 ('2019-01-14', 4, 15142.63),
 ('2019-04-28', 9, 16040.07),
 ('2018-03-08', 9, 15552.73),
 ('2019-07-26', 4, 5935.27),
 ('2019-08-31', 10, 7199.89),
 ('2019-06-15', 3, 7005.52),
 ('2018-03-06', 9, 418.05),
 ('2018-07-09', 6, 16604.27),
 ('2019-08-01', 6, 14588.48),
 ('2019-01-10', 6, 16250.64),
 ('2018-07-09', 4, 18877.6),
 ('2018-01-07', 5, 1224.91),
 ('2019-07-29', 9, 7887.34),
 ('2019-03-30', 1, 7784.71),
 ('2018-11-29', 3, 11236.55),
 ('2018-08-28', 3, 6656.16),
 ('2018-01-19', 9, 17078.02),
 ('2018-03-31', 7, 17725.78),
 ('2019-04-29', 5, 14850.91),
 ('2018-04-17', 1, 18383.69),
 ('2018-07-27', 4, 1141.25),
 ('2018-01-29', 2, 3329.06),
 ('2018-03-02', 7, 10511.41),
 ('2018-12-06', 5, 10951.79),
 ('2019-10-02', 10, 9225.24),
 ('2018-10-29', 4, 15827.36),
 ('2019-10-29', 3, 15452.37),
 ('2018-06-04', 8, 2137.61),
 ('2019-12-10', 4, 15834.32),
 ('2018-05-14', 7, 13077.46),
 ('2018-08-09', 7, 15340.97),
 ('2019-12-03', 4, 9229.17),
 ('2018-01-08', 7, 9517.41),
 ('2018-04-11', 8, 10807.21),
 ('2018-08-16', 2, 7646.83),
 ('2019-09-08', 2, 10388.45),
 ('2018-03-24', 1, 16798.41),
 ('2019-02-15', 1, 16782.76),
 ('2019-09-24', 3, 8212.21),
 ('2019-05-05', 5, 14243.33),
 ('2018-08-06', 4, 16990.09),
 ('2018-07-07', 5, 5190.73),
 ('2018-09-09', 7, 5388.09),
 ('2019-03-30', 3, 19252.2),
 ('2019-08-10', 4, 8870.42),
 ('2019-12-10', 6, 16693.86),
 ('2019-11-18', 9, 2768.18),
 ('2019-12-19', 8, 19314.38),
 ('2019-12-06', 5, 70.09),
 ('2018-07-10', 10, 5604.28),
 ('2019-08-02', 7, 2924.64),
 ('2019-08-06', 6, 13903.23),
 ('2018-03-03', 10, 11433.65),
 ('2018-10-24', 3, 8493.2),
 ('2019-01-04', 6, 19906.9),
 ('2018-07-12', 9, 10004.83),
 ('2018-08-10', 1, 19494.12),
 ('2018-11-20', 5, 12298.77),
 ('2018-06-01', 9, 6777.43),
 ('2018-01-25', 5, 9192.68),
 ('2018-09-29', 1, 8875.31),
 ('2018-08-03', 6, 14972.37),
 ('2018-08-17', 5, 12471.28),
 ('2018-09-21', 10, 18347.73),
 ('2019-08-20', 4, 17245.86),
 ('2019-10-06', 10, 12776.43),
 ('2018-04-02', 1, 16036.43),
 ('2019-10-24', 8, 10127.13),
 ('2018-09-19', 9, 11943.17),
 ('2019-08-22', 4, 6065.74),
 ('2018-05-05', 1, 16390.34),
 ('2019-04-16', 5, 10607.44),
 ('2019-05-21', 10, 5585.3),
 ('2018-12-05', 7, 6699.69),
 ('2019-12-18', 2, 3281.86),
 ('2018-07-23', 1, 9349.78),
 ('2018-07-22', 5, 3500.74),
 ('2018-06-19', 2, 19908.1),
 ('2018-11-16', 7, 10013.38),
 ('2019-11-12', 10, 14537.97),
 ('2019-09-07', 10, 8111.31),
 ('2018-08-19', 9, 6611.63),
 ('2019-05-19', 4, 6477.06),
 ('2019-09-23', 2, 12455.03),
 ('2018-06-23', 9, 2724.89),
 ('2018-09-08', 3, 2264.96),
 ('2018-10-08', 4, 13906.43),
 ('2018-10-10', 10, 16225.96),
 ('2019-06-12', 6, 9746.04),
 ('2018-03-24', 2, 220.91),
 ('2018-02-27', 5, 4976.21),
 ('2019-04-21', 10, 2200.22),
 ('2018-12-07', 3, 1004.32),
 ('2018-03-25', 4, 13784.34),
 ('2018-05-22', 2, 17879.21),
 ('2019-10-04', 7, 4157.62),
 ('2019-08-24', 8, 4558.82),
 ('2019-11-10', 9, 2808.3),
 ('2019-04-13', 5, 15192.68),
 ('2019-10-18', 1, 4024.12),
 ('2018-10-17', 10, 12183.83),
 ('2019-05-23', 8, 4330.82),
 ('2019-04-21', 2, 2672.45),
 ('2018-04-22', 10, 830.84),
 ('2018-01-13', 9, 8941.7),
 ('2018-05-19', 6, 6036.72),
 ('2018-06-03', 2, 18708.42),
 ('2019-11-29', 7, 5402.15),
 ('2019-02-16', 2, 2510.32),
 ('2019-05-22', 4, 763.34),
 ('2019-11-25', 10, 8031.77),
 ('2019-09-17', 2, 17385.13),
 ('2019-08-29', 4, 2374.72),
 ('2019-02-12', 3, 2070.63),
 ('2019-02-14', 1, 5830.89),
 ('2018-07-23', 6, 2439.12),
 ('2018-05-23', 3, 12632.47),
 ('2019-10-15', 8, 5839.02),
 ('2018-04-20', 1, 407.58),
 ('2019-03-19', 8, 18537.33),
 ('2018-07-27', 9, 1382.76),
 ('2018-01-29', 8, 19559.87),
 ('2019-04-28', 5, 1242.92),
 ('2019-12-28', 8, 1046.14),
 ('2019-11-27', 2, 10331.47),
 ('2019-12-01', 4, 8585.08),
 ('2018-01-29', 2, 10059.79),
 ('2018-06-02', 3, 12860.64),
 ('2018-08-31', 6, 14587.51),
 ('2019-07-25', 8, 5127.8),
 ('2019-02-17', 7, 5120.85),
 ('2018-09-15', 1, 14825.88),
 ('2019-11-14', 6, 6342.43),
 ('2018-05-19', 7, 17076.76),
 ('2018-08-12', 1, 17871.65),
 ('2018-08-25', 4, 8958.95),
 ('2019-07-12', 3, 10716.33),
 ('2019-04-07', 10, 5997.9),
 ('2018-03-30', 2, 5498.35),
 ('2019-03-30', 2, 14295.97),
 ('2019-01-03', 10, 15762.22),
 ('2019-09-13', 3, 4396.64),
 ('2019-12-08', 4, 11074.69),
 ('2019-01-28', 8, 14269.82),
 ('2019-09-10', 6, 10193.13),
 ('2019-09-29', 10, 19604.33),
 ('2019-05-10', 10, 15150.77),
 ('2018-08-31', 6, 124.64),
 ('2019-04-21', 4, 18328.71),
 ('2019-05-03', 7, 9107.2),
 ('2018-12-02', 6, 10620.43),
 ('2019-07-18', 5, 16901.83),
 ('2018-05-11', 8, 14001.95),
 ('2019-01-29', 6, 9463.46),
 ('2018-12-27', 6, 7307.84),
 ('2018-02-10', 4, 2174.15),
 ('2019-10-13', 7, 17902.4),
 ('2019-05-31', 2, 18141.49),
 ('2018-06-27', 8, 15727.57),
 ('2018-07-26', 6, 5259.18),
 ('2019-03-21', 7, 6576.86),
 ('2019-01-11', 10, 9482.14),
 ('2019-10-26', 3, 14807.06),
 ('2018-01-19', 10, 5887.29),
 ('2019-10-23', 9, 6727.92),
 ('2018-01-28', 4, 16458.82),
 ('2018-05-24', 10, 17230.22),
 ('2018-09-14', 4, 9132.39),
 ('2018-11-27', 10, 16000.95),
 ('2019-11-08', 1, 15844.9),
 ('2019-05-25', 6, 293.83),
 ('2019-11-17', 8, 4885.2),
 ('2019-09-04', 5, 13225.07),
 ('2019-05-03', 6, 9541.69),
 ('2019-11-22', 9, 11522.79),
 ('2018-10-03', 2, 14989.75),
 ('2019-03-17', 8, 11151.31),
 ('2018-07-16', 8, 13806.56),
 ('2018-01-25', 9, 14739.35),
 ('2018-01-25', 1, 433.06),
 ('2018-05-24', 7, 7392.84),
 ('2019-11-12', 3, 15855.63),
 ('2018-09-06', 2, 13403.38),
 ('2019-04-24', 8, 14174.91),
 ('2018-01-02', 1, 11901.84),
 ('2019-02-26', 3, 19573.61),
 ('2018-04-07', 8, 9123.86),
 ('2019-08-21', 10, 13987.93),
 ('2019-06-06', 3, 6731.15),
 ('2019-12-07', 7, 3434.47),
 ('2018-03-06', 5, 6501.77),
 ('2019-04-09', 8, 16874.87),
 ('2018-11-01', 10, 11667.19),
 ('2019-09-15', 5, 4462.64),
 ('2019-12-08', 8, 12776.72),
 ('2019-04-17', 3, 7304.26),
 ('2019-11-19', 10, 11762.0),
 ('2019-11-07', 1, 2349.25),
 ('2019-05-27', 8, 7444.55),
 ('2019-10-09', 7, 10069.45),
 ('2019-08-25', 8, 13774.08),
 ('2018-04-25', 7, 14045.08),
 ('2019-09-13', 10, 17653.66),
 ('2019-12-24', 3, 4320.13),
 ('2018-10-18', 6, 13098.27),
 ('2019-08-24', 3, 19629.88),
 ('2019-04-05', 4, 7474.86),
 ('2018-04-28', 5, 8726.24),
 ('2018-06-08', 3, 7707.4),
 ('2019-11-15', 6, 1751.97),
 ('2018-12-13', 5, 11647.91),
 ('2018-02-21', 3, 19983.06),
 ('2019-01-13', 8, 19307.06),
 ('2019-11-26', 9, 11168.47),
 ('2019-10-13', 6, 11706.47),
 ('2019-04-20', 6, 4307.41),
 ('2018-10-10', 5, 14376.54),
 ('2018-04-26', 4, 2277.61),
 ('2018-10-06', 10, 6583.53),
 ('2018-03-08', 1, 12472.87),
 ('2018-06-06', 5, 5205.13),
 ('2018-04-02', 3, 4271.59),
 ('2018-11-22', 4, 2824.18),
 ('2018-08-29', 5, 8741.94),
 ('2019-10-27', 6, 19706.15),
 ('2018-05-26', 10, 18770.82),
 ('2018-05-06', 2, 7428.72),
 ('2019-03-10', 6, 12606.33),
 ('2018-02-02', 7, 19687.96),
 ('2018-04-16', 5, 19488.01),
 ('2018-08-06', 1, 14681.39),
 ('2018-04-03', 7, 15911.61),
 ('2018-07-26', 7, 15081.78),
 ('2018-03-14', 2, 9419.01),
 ('2019-01-28', 3, 4674.09),
 ('2019-10-19', 7, 2538.99),
 ('2019-10-05', 7, 16385.46),
 ('2019-05-29', 6, 6437.79),
 ('2019-05-22', 8, 11139.6),
 ('2018-09-30', 4, 18026.42),
 ('2018-05-20', 9, 9982.39),
 ('2019-02-11', 10, 11473.84),
 ('2018-07-08', 6, 12226.83),
 ('2018-06-30', 10, 8810.12),
 ('2019-06-21', 3, 19879.07),
 ('2018-01-11', 7, 2501.68),
 ('2019-05-22', 1, 7297.54),
 ('2019-11-07', 2, 14689.22),
 ('2018-02-04', 1, 3305.66),
 ('2018-12-14', 4, 7659.96),
 ('2019-07-10', 7, 5182.42),
 ('2019-09-18', 3, 10053.03),
 ('2018-12-01', 9, 5148.49),
 ('2019-06-16', 10, 2846.56),
 ('2018-02-25', 5, 95.91),
 ('2018-02-22', 3, 9334.69),
 ('2018-07-20', 8, 12677.09),
 ('2019-02-08', 6, 11279.12),
 ('2019-08-16', 8, 2317.02),
 ('2018-02-03', 4, 14219.78),
 ('2019-02-03', 1, 4574.72),
 ('2018-03-31', 6, 735.35),
 ('2018-07-26', 5, 8647.85),
 ('2018-10-12', 6, 5508.82),
 ('2018-06-04', 8, 16110.72),
 ('2019-08-23', 8, 2944.73),
 ('2019-03-31', 7, 458.01),
 ('2019-10-27', 7, 17629.04),
 ('2018-11-13', 10, 8937.0),
 ('2018-03-01', 10, 7593.75),
 ('2018-02-20', 1, 2855.01),
 ('2018-09-24', 9, 17643.96),
 ('2018-03-06', 4, 2457.05),
 ('2019-04-05', 3, 18553.49),
 ('2019-06-17', 8, 19301.54),
 ('2018-08-18', 1, 154.94),
 ('2019-05-04', 7, 2826.98),
 ('2019-01-06', 7, 5117.55),
 ('2018-04-03', 6, 6380.73),
 ('2018-06-03', 3, 13634.75),
 ('2019-06-04', 4, 13086.32),
 ('2019-12-01', 7, 463.48),
 ('2018-12-24', 7, 3870.31),
 ('2019-03-04', 5, 11200.38),
 ('2019-05-11', 6, 16101.26),
 ('2019-10-18', 6, 1420.3),
 ('2018-01-22', 7, 1707.91),
 ('2018-01-26', 9, 18928.92),
 ('2019-05-09', 2, 1928.62),
 ('2018-08-09', 2, 10309.14),
 ('2018-09-13', 1, 346.47),
 ('2019-12-12', 10, 4558.08),
 ('2018-11-30', 3, 4202.93),
 ('2018-02-03', 3, 2772.56),
 ('2019-10-29', 3, 11715.6),
 ('2018-09-06', 1, 17456.38),
 ('2018-05-07', 2, 8486.53),
 ('2018-01-24', 2, 15883.72),
 ('2018-12-14', 3, 17896.74),
 ('2018-04-04', 1, 18875.85),
 ('2018-08-28', 7, 11222.39),
 ('2019-02-04', 2, 18306.97),
 ('2018-07-09', 4, 11458.99),
 ('2018-02-12', 7, 17803.8),
 ('2018-09-26', 8, 10866.39),
 ('2018-07-02', 2, 12718.83),
 ('2019-06-08', 9, 8742.85),
 ('2019-06-19', 5, 10005.38),
 ('2019-04-19', 3, 9930.67),
 ('2019-11-19', 5, 14577.02),
 ('2018-06-22', 10, 14131.43),
 ('2018-01-26', 9, 6952.73),
 ('2018-05-24', 9, 11841.34),
 ('2019-10-10', 3, 8313.17),
 ('2019-11-29', 1, 6721.59),
 ('2018-10-27', 8, 15047.04),
 ('2018-11-25', 2, 6261.19),
 ('2018-12-07', 5, 16334.43),
 ('2019-05-02', 8, 11203.72),
 ('2019-08-01', 7, 12467.44),
 ('2019-07-14', 10, 9791.71),
 ('2018-09-01', 2, 4220.71),
 ('2018-03-21', 6, 19297.86),
 ('2019-02-24', 5, 1598.15),
 ('2018-11-28', 8, 19244.15),
 ('2018-01-30', 3, 18585.35),
 ('2019-01-20', 10, 19277.14),
 ('2019-11-14', 3, 4394.95),
 ('2018-11-17', 7, 11453.96),
 ('2019-06-16', 10, 13346.86),
 ('2018-10-21', 1, 14961.42),
 ('2018-12-29', 3, 10698.74),
 ('2019-07-04', 3, 11874.12),
 ('2018-11-20', 9, 11337.19),
 ('2019-10-16', 3, 16802.31),
 ('2019-05-11', 1, 11864.64),
 ('2018-05-18', 4, 11470.69),
 ('2019-06-06', 5, 5740.87),
 ('2018-09-18', 5, 7516.74),
 ('2019-05-27', 9, 5219.97),
 ('2018-06-14', 5, 14452.63),
 ('2018-03-25', 2, 7074.08),
 ('2018-09-03', 2, 5081.52),
 ('2018-04-16', 7, 11322.95),
 ('2019-06-24', 3, 13088.55),
 ('2019-05-17', 4, 13191.87),
 ('2018-09-18', 7, 9843.16),
 ('2019-12-01', 7, 9521.16),
 ('2019-04-24', 4, 4310.91),
 ('2019-05-07', 3, 5138.0),
 ('2019-06-25', 2, 12112.63),
 ('2018-12-26', 10, 4214.38),
 ('2019-08-11', 8, 11880.67),
 ('2018-07-18', 6, 4988.86),
 ('2019-04-07', 2, 3612.25),
 ('2019-08-13', 1, 17566.59),
 ('2018-04-19', 5, 19843.7),
 ('2019-02-04', 1, 929.74),
 ('2018-09-12', 5, 924.17),
 ('2018-02-08', 2, 12882.27),
 ('2019-03-17', 10, 16938.15),
 ('2018-11-28', 1, 17921.69),
 ('2019-01-18', 9, 2388.51),
 ('2018-08-22', 8, 6884.34),
 ('2018-12-11', 6, 17364.11),
 ('2018-01-10', 5, 16807.44),
 ('2018-09-22', 8, 12745.78),
 ('2019-03-16', 6, 13617.09),
 ('2019-10-07', 4, 8029.82),
 ('2019-02-11', 9, 15905.9),
 ('2019-02-19', 4, 9394.62),
 ('2019-06-18', 9, 14051.43),
 ('2019-05-22', 2, 5493.66),
 ('2018-09-07', 6, 4760.66),
 ('2018-10-31', 5, 18712.52),
 ('2018-07-14', 5, 11144.42),
 ('2019-12-13', 9, 5544.46),
 ('2019-09-16', 6, 4351.69),
 ('2018-09-06', 1, 12039.39),
 ('2019-03-05', 8, 4938.61),
 ('2018-02-18', 6, 2909.54),
 ('2019-11-11', 3, 4131.28),
 ('2018-09-13', 9, 11922.12),
 ('2018-10-07', 2, 12448.0),
 ('2019-09-03', 7, 7846.1),
 ('2018-02-06', 1, 5796.91),
 ('2018-06-09', 7, 2918.08),
 ('2019-04-04', 4, 14486.74),
 ('2018-04-04', 1, 2396.23),
 ('2018-10-02', 4, 16654.53),
 ('2018-11-29', 7, 8185.04),
 ('2019-03-09', 9, 6566.6),
 ('2018-06-27', 2, 10812.77),
 ('2018-06-13', 2, 7990.76),
 ('2019-02-01', 3, 889.26),
 ('2019-12-07', 7, 9662.47),
 ('2018-12-13', 2, 6830.09),
 ('2018-03-12', 2, 10219.79),
 ('2018-07-20', 5, 2994.8),
 ('2019-09-07', 8, 17331.9),
 ('2018-09-02', 7, 6354.14),
 ('2018-05-23', 2, 7748.04),
 ('2018-08-30', 10, 18656.65),
 ('2018-12-29', 5, 14552.27),
 ('2019-10-28', 8, 811.15),
 ('2018-07-03', 2, 3616.99),
 ('2019-10-24', 10, 15948.69),
 ('2019-08-09', 10, 14968.36),
 ('2019-10-10', 6, 18681.22),
 ('2019-01-24', 8, 11735.97),
 ('2018-05-16', 10, 10044.5),
 ('2018-12-06', 9, 13661.03),
 ('2019-04-12', 10, 17249.2),
 ('2018-10-13', 4, 18606.16),
 ('2018-02-05', 5, 15505.57),
 ('2019-09-03', 2, 15947.57),
 ('2018-03-26', 5, 15842.38),
 ('2018-11-19', 5, 14045.05),
 ('2019-03-03', 10, 2546.99),
 ('2018-02-13', 9, 9265.37),
 ('2018-05-10', 2, 4217.42),
 ('2018-04-19', 6, 5409.73),
 ('2018-01-11', 4, 6986.0),
 ('2018-07-05', 4, 6796.79),
 ('2018-06-29', 8, 19497.0),
 ('2018-08-17', 2, 312.19),
 ('2019-12-23', 9, 13864.56),
 ('2018-01-16', 9, 17839.56),
 ('2019-06-30', 7, 7294.39),
 ('2018-05-15', 3, 16428.71),
 ('2018-04-12', 4, 2855.53),
 ('2019-03-10', 4, 10401.47),
 ('2019-06-07', 5, 5155.45),
 ('2019-04-21', 1, 2899.8),
 ('2019-10-11', 4, 6840.77),
 ('2019-02-11', 5, 5055.71),
 ('2019-12-29', 2, 14909.77),
 ('2018-05-29', 4, 8961.58),
 ('2018-06-26', 3, 2991.89),
 ('2018-09-04', 9, 2673.58),
 ('2018-09-17', 1, 16148.23),
 ('2019-05-29', 10, 1954.14),
 ('2018-04-28', 9, 2184.82),
 ('2018-12-30', 9, 15729.07),
 ('2019-03-13', 5, 11976.38),
 ('2018-12-16', 8, 19755.81),
 ('2019-05-03', 5, 18227.24),
 ('2019-11-11', 1, 17980.01),
 ('2018-03-20', 7, 6554.3),
 ('2018-09-14', 10, 3955.2),
 ('2019-05-13', 2, 16933.09),
 ('2019-09-22', 6, 1421.23),
 ('2019-11-30', 4, 201.7),
 ('2019-09-21', 1, 4367.17),
 ('2018-08-24', 9, 1173.07),
 ('2019-07-25', 5, 5669.26),
 ('2018-02-13', 9, 6679.02),
 ('2018-07-12', 10, 3168.83),
 ('2018-09-01', 8, 14501.82),
 ('2019-06-07', 2, 16252.97),
 ('2018-05-21', 6, 10677.73),
 ('2018-07-16', 10, 3440.64),
 ('2018-03-14', 5, 10579.88),
 ('2019-12-10', 6, 17299.97),
 ('2018-06-06', 3, 4426.61),
 ('2019-05-30', 1, 11436.49),
 ('2018-05-09', 5, 1273.51),
 ('2018-08-05', 10, 17980.77),
 ('2018-09-20', 9, 13485.64),
 ('2018-03-29', 9, 14697.31),
 ('2018-07-10', 9, 12769.13),
 ('2018-01-04', 2, 17993.5),
 ('2019-12-16', 7, 17469.81),
 ('2019-01-21', 7, 9311.21),
 ('2018-04-13', 4, 3908.12),
 ('2019-10-10', 2, 18063.54),
 ('2018-02-26', 10, 2590.57),
 ('2018-03-13', 8, 4110.27),
 ('2018-01-19', 7, 18279.91),
 ('2019-03-04', 2, 19858.34),
 ('2018-08-17', 10, 19889.94),
 ('2018-01-15', 7, 19002.55),
 ('2019-10-30', 4, 18151.76),
 ('2018-10-16', 10, 13021.75),
 ('2019-10-14', 2, 17224.19),
 ('2018-04-04', 2, 7575.61),
 ('2019-09-23', 1, 6318.66),
 ('2018-01-17', 3, 8123.92),
 ('2018-04-07', 10, 3122.65),
 ('2018-03-16', 4, 18019.45),
 ('2019-03-08', 2, 8271.26),
 ('2018-12-17', 6, 10851.55),
 ('2019-03-29', 7, 2993.82),
 ('2018-05-07', 6, 3637.25),
 ('2018-06-19', 2, 18475.05),
 ('2018-08-16', 8, 15322.67),
 ('2019-11-13', 1, 13150.18),
 ('2018-01-14', 5, 18563.24),
 ('2018-06-02', 7, 18739.44),
 ('2018-01-31', 10, 14431.82),
 ('2018-04-03', 5, 2093.52),
 ('2018-08-10', 5, 5096.92),
 ('2018-02-20', 9, 16332.78),
 ('2018-04-09', 3, 19212.44),
 ('2019-04-04', 7, 4129.38),
 ('2018-06-10', 4, 3576.11),
 ('2019-12-25', 1, 18426.93),
 ('2018-01-25', 6, 18874.98),
 ('2018-10-12', 1, 954.34),
 ('2018-07-24', 1, 9663.4),
 ('2018-08-26', 7, 11280.06),
 ('2019-04-29', 10, 7758.82),
 ('2018-09-19', 2, 6270.74),
 ('2018-03-12', 4, 1752.8),
 ('2018-08-05', 7, 16962.95),
 ('2018-10-26', 5, 19896.83),
 ('2018-09-01', 8, 11669.85),
 ('2018-01-28', 10, 6769.59),
 ('2018-09-08', 4, 1575.43),
 ('2018-09-25', 1, 12538.51),
 ('2018-09-18', 5, 12366.06),
 ('2019-05-07', 7, 8132.26),
 ('2019-02-24', 9, 805.63),
 ('2018-03-22', 1, 2503.89),
 ('2018-12-20', 4, 17969.65),
 ('2018-09-13', 6, 141.93),
 ('2019-02-21', 5, 4202.8),
 ('2019-10-17', 1, 6845.75),
 ('2019-09-13', 9, 13990.42),
 ('2018-11-13', 10, 9032.81),
 ('2018-03-22', 5, 18808.38),
 ('2018-03-23', 5, 3654.81),
 ('2018-11-11', 3, 14761.44)]
len(Ventas)

1000

In [18]:
Productos = [(1, 'Descripcion 1', 'Calzado'),
 (2, 'Descripcion 2', 'Alimentos'),
 (3, 'Descripcion 3', 'Jardineria'),
 (4, 'Descripcion 4', 'Detailing'),
 (5, 'Descripcion 5', 'Calzado'),
 (6, 'Descripcion 6', 'Jardineria'),
 (7, 'Descripcion 7', 'Limpieza'),
 (8, 'Descripcion 8', 'Detailing'),
 (9, 'Descripcion 9', 'Alimentos'),
 (10, 'Descripcion 10', 'Detailing')]

In [19]:
VentaDeProductos = sc.parallelize(Ventas, 8)
DetallesProducto = sc.parallelize(Productos, 8)

### A) Cuál es el producto más vendido.

In [20]:
productos = VentaDeProductos.map(lambda x: (x[1], 1))
productos.take(5)

[(9, 1), (5, 1), (6, 1), (9, 1), (9, 1)]

In [21]:
productos.reduceByKey(lambda a,b: a+b).reduce(lambda x,y: x if x[1]>y[1] else y)


(7, 111)

### B) Cuál es la categoría de productos más vendida.

In [22]:
Categorias = DetallesProducto.map(lambda x: (x[0],x[2]))
Categorias.collect()

[(1, 'Calzado'),
 (2, 'Alimentos'),
 (3, 'Jardineria'),
 (4, 'Detailing'),
 (5, 'Calzado'),
 (6, 'Jardineria'),
 (7, 'Limpieza'),
 (8, 'Detailing'),
 (9, 'Alimentos'),
 (10, 'Detailing')]

In [23]:
Resultado = VentaDeProductos.map(lambda x: (x[1],x[0]))
Resultado.take(5)

[(9, '2018-08-17'),
 (5, '2018-01-05'),
 (6, '2018-08-26'),
 (9, '2018-12-31'),
 (9, '2018-05-26')]

In [24]:
ResultadoJoin = Resultado.join(Categorias)
ResultadoJoin.take(5)

[(1, ('2018-07-01', 'Calzado')),
 (1, ('2019-04-09', 'Calzado')),
 (1, ('2019-10-13', 'Calzado')),
 (1, ('2018-06-25', 'Calzado')),
 (1, ('2019-05-06', 'Calzado'))]

In [25]:
Resultado = ResultadoJoin.map(lambda x: (x[1][1],1))
Resultado.take(5)

[('Calzado', 1),
 ('Calzado', 1),
 ('Calzado', 1),
 ('Calzado', 1),
 ('Calzado', 1)]

In [26]:
Resultado1 = Resultado.reduceByKey(lambda a,b: a+b)
Resultado1.collect()

[('Calzado', 206),
 ('Detailing', 290),
 ('Limpieza', 111),
 ('Alimentos', 202),
 ('Jardineria', 191)]

In [27]:
Resultado1.reduce(lambda a,b: a if a[1]>b[1] else b)

('Detailing', 290)

### C) Cuál es el top5 de productos más vendidos generando un RDD con (código de producto, descripción, cantidad de ventas)

In [28]:
productosmasvendidos = productos.reduceByKey(lambda a,b: a+b)
productosmasvendidos.take(10)

[(8, 88),
 (9, 102),
 (1, 107),
 (2, 100),
 (10, 98),
 (3, 93),
 (4, 104),
 (5, 99),
 (6, 98),
 (7, 111)]

In [29]:
DesCodigo = DetallesProducto.map(lambda x: (x[0],x[1]))
DesCodigo.take(10)

[(1, 'Descripcion 1'),
 (2, 'Descripcion 2'),
 (3, 'Descripcion 3'),
 (4, 'Descripcion 4'),
 (5, 'Descripcion 5'),
 (6, 'Descripcion 6'),
 (7, 'Descripcion 7'),
 (8, 'Descripcion 8'),
 (9, 'Descripcion 9'),
 (10, 'Descripcion 10')]

In [30]:
productosmasvendidos_exten = productosmasvendidos.join(DesCodigo)
productosmasvendidos_exten.take(10)

[(1, (107, 'Descripcion 1')),
 (2, (100, 'Descripcion 2')),
 (3, (93, 'Descripcion 3')),
 (4, (104, 'Descripcion 4')),
 (5, (99, 'Descripcion 5')),
 (6, (98, 'Descripcion 6')),
 (7, (111, 'Descripcion 7')),
 (8, (88, 'Descripcion 8')),
 (9, (102, 'Descripcion 9')),
 (10, (98, 'Descripcion 10'))]

In [31]:
productosmasvendidos_exten.map(lambda x: (x[0],x[1][1],x[1][0])).takeOrdered(5, key=lambda x: -x[2])

[(7, 'Descripcion 7', 111),
 (1, 'Descripcion 1', 107),
 (4, 'Descripcion 4', 104),
 (9, 'Descripcion 9', 102),
 (2, 'Descripcion 2', 100)]

### D) Cuál es el producto que registró mayor aumento de precio en el último año, tomando para este análisis solo los productos que cuenten con al menos 50 ventas en el último año.

In [32]:
FiltroAño = VentaDeProductos.filter(lambda x: x[0] >= '2019-06-06').cache()
FiltroAño.take(5)

[('2019-12-03', 4, 17657.29),
 ('2019-09-06', 2, 5457.19),
 ('2019-07-12', 4, 10859.8),
 ('2019-10-13', 1, 15204.76),
 ('2019-10-25', 4, 4385.27)]

In [33]:
Ordenado = FiltroAño.map(lambda x: (x[1],x[2]))
Ordenado.take(5)

[(4, 17657.29), (2, 5457.19), (4, 10859.8), (1, 15204.76), (4, 4385.27)]

In [34]:
PrecioMin = Ordenado.reduceByKey(lambda a,b: a if a<b else b)
PrecioMin.take(5)

[(8, 811.15), (1, 2349.25), (9, 1091.59), (2, 105.97), (10, 213.97)]

In [35]:
PrecioMax = Ordenado.reduceByKey(lambda a,b: a if a>b else b)
PrecioMax.take(5)

[(8, 19918.22), (1, 19405.53), (9, 19614.34), (2, 19158.44), (10, 19604.33)]

In [36]:
Precios = PrecioMin.join(PrecioMax)
Precios.take(5)

[(8, (811.15, 19918.22)),
 (1, (2349.25, 19405.53)),
 (9, (1091.59, 19614.34)),
 (2, (105.97, 19158.44)),
 (10, (213.97, 19604.33))]

In [37]:
Mas25Ventas = Ordenado.map(lambda x: (x[0],1))
Pepe= Mas25Ventas.reduceByKey(lambda a,b: a+b)
Pepe.take(5)

[(8, 27), (1, 27), (9, 24), (2, 22), (10, 28)]

In [38]:
Ventas25 = Pepe.filter(lambda x: x[1] >= 25 )
Ventas25.collect()

[(8, 27), (1, 27), (10, 28), (3, 29), (4, 26), (6, 28), (7, 30)]

In [39]:
CumplenRequisitos = Ventas25.leftOuterJoin(Precios)
CumplenRequisitos.collect()

[(8, (27, (811.15, 19918.22))),
 (1, (27, (2349.25, 19405.53))),
 (10, (28, (213.97, 19604.33))),
 (3, (29, (1223.33, 19880.2))),
 (4, (26, (201.7, 19974.25))),
 (6, (28, (889.75, 19706.15))),
 (7, (30, (463.48, 18206.0)))]

In [40]:
MayoresAumentos = CumplenRequisitos.map(lambda x: (x[0],x[1][1][0], x[1][1][1] ))
MayoresAumentos.collect()

[(8, 811.15, 19918.22),
 (1, 2349.25, 19405.53),
 (10, 213.97, 19604.33),
 (3, 1223.33, 19880.2),
 (4, 201.7, 19974.25),
 (6, 889.75, 19706.15),
 (7, 463.48, 18206.0)]

In [41]:
MayoresAumentos.reduce(lambda x,y: x if x[2]*100/x[1] > y[2]*100/y[1] else y)

(4, 201.7, 19974.25)

### E) Idem anterior, pero calculando la categoría de productos que registró mayor variación de precios en el último año.

In [42]:
Categorias = DetallesProducto.map(lambda x: (x[0],x[2]))
Categorias.collect()

[(1, 'Calzado'),
 (2, 'Alimentos'),
 (3, 'Jardineria'),
 (4, 'Detailing'),
 (5, 'Calzado'),
 (6, 'Jardineria'),
 (7, 'Limpieza'),
 (8, 'Detailing'),
 (9, 'Alimentos'),
 (10, 'Detailing')]

In [43]:
FiltroAño.take(5)

[('2019-12-03', 4, 17657.29),
 ('2019-09-06', 2, 5457.19),
 ('2019-07-12', 4, 10859.8),
 ('2019-10-13', 1, 15204.76),
 ('2019-10-25', 4, 4385.27)]

In [44]:
join = FiltroAño.map(lambda x: (x[1],x[2])).join(Categorias).map(lambda x: (x[1][1],(x[1][0],1)))
join.take(5)

[('Calzado', (15204.76, 1)),
 ('Calzado', (3957.78, 1)),
 ('Calzado', (5322.71, 1)),
 ('Calzado', (4211.18, 1)),
 ('Calzado', (7644.12, 1))]

In [45]:
categorias_mas_28ventas = join.reduceByKey(lambda x,y: (x[1]+y[1],x[1]+y[1])).map(lambda x: (x[0], x[1][0])).filter(lambda x: x[1]>48)
categorias_mas_28ventas.take(5)

[('Calzado', 49), ('Detailing', 81), ('Jardineria', 57)]

In [46]:
join = FiltroAño.map(lambda x: (x[1],x[2])).join(Categorias).map(lambda x: (x[1][1],x[1][0]))
join.take(5)

[('Calzado', 15204.76),
 ('Calzado', 3957.78),
 ('Calzado', 5322.71),
 ('Calzado', 4211.18),
 ('Calzado', 7644.12)]

In [47]:
categoria_max = join.reduceByKey(lambda x,y: x if x > y else y)
categoria_max.take(5)

[('Calzado', 19405.53),
 ('Detailing', 19974.25),
 ('Limpieza', 18206.0),
 ('Alimentos', 19614.34),
 ('Jardineria', 19880.2)]

In [48]:
categoria_min = join.reduceByKey(lambda x,y: x if x < y else y)
categoria_min.take(5)

[('Calzado', 70.09),
 ('Detailing', 201.7),
 ('Limpieza', 463.48),
 ('Alimentos', 105.97),
 ('Jardineria', 889.75)]

In [49]:
categoria_precio = categoria_max.join(categoria_min)
categoria_precio.take(5)

[('Calzado', (19405.53, 70.09)),
 ('Detailing', (19974.25, 201.7)),
 ('Limpieza', (18206.0, 463.48)),
 ('Alimentos', (19614.34, 105.97)),
 ('Jardineria', (19880.2, 889.75))]

In [50]:
final = categoria_precio.rightOuterJoin(categorias_mas_28ventas).map(lambda x: (x[0],x[1][0][0],x[1][0][1]))
final.reduce(lambda x,y: x if x[2]*100/x[1] > y[2]*100/y[1] else y)

('Jardineria', 19880.2, 889.75)

## 3) Se tiene un RDD con información de vuelos programados con la forma (número de vuelo, código de aerolínea, código de aeropuerto de salida, código de aeropuerto de llegada, fecha de salida AAAAMMDD, hora de salida HH:MM, fecha de llegada AAAAMMDD, hora de llegada HH:MM). A su vez, se cuenta con el registro actualizado del estado de los vuelos que fueron ocurriendo, con la forma (número de vuelo, aerolínea, fecha de salida AAAAMMDD, hora de salida HH:MM, fecha de llegada AAAAMMDD, hora de llegada HH:MM, estado). En base al estado, podría contar con algún dato en blanco, por ejemplo si el vuelo fue cancelado no tendrá información de fechas y horas, si el vuelo se encuentra aún en curso, no contendrá información de la llegada. Se pide resolver utilizando PySpark:

In [51]:
import random
from datetime import *

In [52]:
#(número de vuelo, código de aerolínea, código de aeropuerto de salida, código de aeropuerto de llegada, fecha de salida AAAAMMDD, hora de salida HH:MM, fecha de llegada AAAAMMDD, hora de llegada HH:MM)
vuelos =  [(1, 2, 'AEP', 'MDQ', '20200320', '06:15', '20200320', '07:05'),
 (2, 1, 'AEP', 'CPC', '20200320', '12:00', '20200320', '14:15'),
 (3, 3, 'BRC', 'EZE', '20200320', '17:15', '20200320', '19:30'),
 (4, 3, 'AEP', 'IGR', '20200320', '21:00', '20200320', '22:40'),
 (5, 3, 'MDQ', 'IGR', '20200321', '06:00', '20200321', '08:00'),
 (6, 1, 'IGR', 'USH', '20200321', '21:00', '20200322', '04:20'),
 (7, 3, 'USH', 'AEP', '20200322', '15:15', '20200322', '18:20'),
 (8, 2, 'AEP', 'RGA', '20200322', '17:00', '20200322', '20:15'),
 (9, 3, 'BHI', 'COR', '20200323', '09:10', '20200323', '14:00'),
 (10, 2, 'NQN', 'BRC', '20200323', '14:45', '20200323', '15:05')]

#(número de vuelo, aerolínea, fecha de salida AAAAMMDD, hora de salida HH:MM, fecha de llegada AAAAMMDD, hora de llegada HH:MM, estado)
estados =  [(1, 'LATAM', '20200320', '06:15', '20200320', '07:05', 'Finalizado'),
 (2, 'AMERICAN', '20200320', '12:00', '20200320', '14:28', 'Finalizado'),
 (3, 'IBERIA', '20200320', '17:15', None, None, 'En curso'),
 (4, 'IBERIA', None, None, None, None, 'Cancelado'),
 (5, 'IBERIA', '20200321', '06:23', '20200321', '08:30', 'Finalizado'),
 (6, 'AMERICAN', None, None, None, None, 'Cancelado'),
 (7, 'IBERIA', '20200322', '15:15', '20200322', '18:20', 'Finalizado'),
 (8, 'LATAM', '20200322', '17:05', None, None, 'En curso'),
 (9, 'IBERIA', None, None, None, None, 'Cancelado'),
 (10, 'LATAM', '20200323', '14:45', '20200323', '15:05', 'Finalizado')]

In [53]:
vuelosRDD = sc.parallelize(vuelos)
vuelosRDD.collect()

[(1, 2, 'AEP', 'MDQ', '20200320', '06:15', '20200320', '07:05'),
 (2, 1, 'AEP', 'CPC', '20200320', '12:00', '20200320', '14:15'),
 (3, 3, 'BRC', 'EZE', '20200320', '17:15', '20200320', '19:30'),
 (4, 3, 'AEP', 'IGR', '20200320', '21:00', '20200320', '22:40'),
 (5, 3, 'MDQ', 'IGR', '20200321', '06:00', '20200321', '08:00'),
 (6, 1, 'IGR', 'USH', '20200321', '21:00', '20200322', '04:20'),
 (7, 3, 'USH', 'AEP', '20200322', '15:15', '20200322', '18:20'),
 (8, 2, 'AEP', 'RGA', '20200322', '17:00', '20200322', '20:15'),
 (9, 3, 'BHI', 'COR', '20200323', '09:10', '20200323', '14:00'),
 (10, 2, 'NQN', 'BRC', '20200323', '14:45', '20200323', '15:05')]

In [54]:
estadosRDD = sc.parallelize(estados)
estadosRDD.collect()

[(1, 'LATAM', '20200320', '06:15', '20200320', '07:05', 'Finalizado'),
 (2, 'AMERICAN', '20200320', '12:00', '20200320', '14:28', 'Finalizado'),
 (3, 'IBERIA', '20200320', '17:15', None, None, 'En curso'),
 (4, 'IBERIA', None, None, None, None, 'Cancelado'),
 (5, 'IBERIA', '20200321', '06:23', '20200321', '08:30', 'Finalizado'),
 (6, 'AMERICAN', None, None, None, None, 'Cancelado'),
 (7, 'IBERIA', '20200322', '15:15', '20200322', '18:20', 'Finalizado'),
 (8, 'LATAM', '20200322', '17:05', None, None, 'En curso'),
 (9, 'IBERIA', None, None, None, None, 'Cancelado'),
 (10, 'LATAM', '20200323', '14:45', '20200323', '15:05', 'Finalizado')]

### A) Cuál es el aeropuerto con mayor tránsito.

In [55]:
AeropuertosProgramados = vuelosRDD.map(lambda x: (x[0], (x[2], x[3])) )
AeropuertosProgramados.collect()

[(1, ('AEP', 'MDQ')),
 (2, ('AEP', 'CPC')),
 (3, ('BRC', 'EZE')),
 (4, ('AEP', 'IGR')),
 (5, ('MDQ', 'IGR')),
 (6, ('IGR', 'USH')),
 (7, ('USH', 'AEP')),
 (8, ('AEP', 'RGA')),
 (9, ('BHI', 'COR')),
 (10, ('NQN', 'BRC'))]

In [56]:
SoloEstados = estadosRDD.map(lambda x: (x[0], x[6]))
SoloEstados.collect()

[(1, 'Finalizado'),
 (2, 'Finalizado'),
 (3, 'En curso'),
 (4, 'Cancelado'),
 (5, 'Finalizado'),
 (6, 'Cancelado'),
 (7, 'Finalizado'),
 (8, 'En curso'),
 (9, 'Cancelado'),
 (10, 'Finalizado')]

In [57]:
VuelosRealizados = SoloEstados.filter(lambda x: (x[1] == 'Finalizado'))
VuelosRealizados.collect()

[(1, 'Finalizado'),
 (2, 'Finalizado'),
 (5, 'Finalizado'),
 (7, 'Finalizado'),
 (10, 'Finalizado')]

In [58]:
VuelosFinalizados = VuelosRealizados.leftOuterJoin(AeropuertosProgramados)
VuelosFinalizados.collect()

[(1, ('Finalizado', ('AEP', 'MDQ'))),
 (5, ('Finalizado', ('MDQ', 'IGR'))),
 (2, ('Finalizado', ('AEP', 'CPC'))),
 (10, ('Finalizado', ('NQN', 'BRC'))),
 (7, ('Finalizado', ('USH', 'AEP')))]

In [59]:
FrecuenciaSalida = VuelosFinalizados.map(lambda x: (x[1][1][0], 1) )
FrecuenciaSalida.collect()

[('AEP', 1), ('MDQ', 1), ('AEP', 1), ('NQN', 1), ('USH', 1)]

In [60]:
FrecS = FrecuenciaSalida.reduceByKey(lambda a,b: a+b)
FrecS.collect()

[('AEP', 2), ('MDQ', 1), ('NQN', 1), ('USH', 1)]

In [61]:
FrecuenciaLlegada = VuelosFinalizados.map(lambda x: (x[1][1][1], 1) )
FrecuenciaLlegada.collect()

[('MDQ', 1), ('IGR', 1), ('CPC', 1), ('BRC', 1), ('AEP', 1)]

In [62]:
FrecL = FrecuenciaLlegada.reduceByKey(lambda a,b: a+b)
FrecL.collect()

[('MDQ', 1), ('AEP', 1), ('IGR', 1), ('CPC', 1), ('BRC', 1)]

In [63]:
Total = FrecS.fullOuterJoin(FrecL)
Total.collect()

[('AEP', (2, 1)),
 ('MDQ', (1, 1)),
 ('NQN', (1, None)),
 ('USH', (1, None)),
 ('IGR', (None, 1)),
 ('CPC', (None, 1)),
 ('BRC', (None, 1))]

In [64]:
Resultado = Total.map( lambda x:(x[0], (0 if x[1][0] == None else x[1][0]),(0 if x[1][1] == None else x[1][1])))
Resultado.collect()

[('AEP', 2, 1),
 ('MDQ', 1, 1),
 ('NQN', 1, 0),
 ('USH', 1, 0),
 ('IGR', 0, 1),
 ('CPC', 0, 1),
 ('BRC', 0, 1)]

In [65]:
MasTransito = Resultado.map(lambda x: (x[0], x[1]+x[2]) )
MasTransito.reduce(lambda a,b: a if a[1]>b[1] else b)

('AEP', 3)

### B) Cuál es la aerolínea con mayor cantidad de vuelos.

In [66]:
estadosRDD.collect()

[(1, 'LATAM', '20200320', '06:15', '20200320', '07:05', 'Finalizado'),
 (2, 'AMERICAN', '20200320', '12:00', '20200320', '14:28', 'Finalizado'),
 (3, 'IBERIA', '20200320', '17:15', None, None, 'En curso'),
 (4, 'IBERIA', None, None, None, None, 'Cancelado'),
 (5, 'IBERIA', '20200321', '06:23', '20200321', '08:30', 'Finalizado'),
 (6, 'AMERICAN', None, None, None, None, 'Cancelado'),
 (7, 'IBERIA', '20200322', '15:15', '20200322', '18:20', 'Finalizado'),
 (8, 'LATAM', '20200322', '17:05', None, None, 'En curso'),
 (9, 'IBERIA', None, None, None, None, 'Cancelado'),
 (10, 'LATAM', '20200323', '14:45', '20200323', '15:05', 'Finalizado')]

In [67]:
Vuelos = estadosRDD.map(lambda x: (x[1],1)).reduceByKey(lambda a,b : a+b)
Vuelos.reduce(lambda a,b: a if a[1]>b[1] else b )

('IBERIA', 5)

### C) Cuál es la aerolínea con mayor cantidad de cancelaciones.

In [68]:
Cancelaciones = estadosRDD.filter(lambda x: x[6]== 'Cancelado').map(lambda x: (x[1],1)).reduceByKey(lambda a,b : a+b)
Cancelaciones.reduce(lambda a,b: a if a[1]>b[1] else b )

('IBERIA', 2)

### D) Cuál es el vuelo (numero de vuelo + fecha) con mayor retraso en el horario de salida.

In [69]:
VuelosProgramadosRetraso = vuelosRDD.map(lambda x: (x[0], (x[5], x[4])))
VuelosProgramadosRetraso.collect()

[(1, ('06:15', '20200320')),
 (2, ('12:00', '20200320')),
 (3, ('17:15', '20200320')),
 (4, ('21:00', '20200320')),
 (5, ('06:00', '20200321')),
 (6, ('21:00', '20200321')),
 (7, ('15:15', '20200322')),
 (8, ('17:00', '20200322')),
 (9, ('09:10', '20200323')),
 (10, ('14:45', '20200323'))]

In [70]:
EstadosRetrasoFiltrado = estadosRDD.filter(lambda x: x[6] == 'Finalizado')
EstadosRetraso =  EstadosRetrasoFiltrado.map(lambda x: (x[0],x[3]))
EstadosRetraso.collect()

[(1, '06:15'), (2, '12:00'), (5, '06:23'), (7, '15:15'), (10, '14:45')]

In [71]:
Realizados = EstadosRetraso.leftOuterJoin(VuelosProgramadosRetraso)
Realizados.collect()

[(1, ('06:15', ('06:15', '20200320'))),
 (5, ('06:23', ('06:00', '20200321'))),
 (2, ('12:00', ('12:00', '20200320'))),
 (10, ('14:45', ('14:45', '20200323'))),
 (7, ('15:15', ('15:15', '20200322')))]

In [72]:
Pepe = Realizados.map(lambda x: (x[0],x[1][0] ,x[1][1][0], x[1][1][1]))
Pepe.collect()

[(1, '06:15', '06:15', '20200320'),
 (5, '06:23', '06:00', '20200321'),
 (2, '12:00', '12:00', '20200320'),
 (10, '14:45', '14:45', '20200323'),
 (7, '15:15', '15:15', '20200322')]

In [73]:
Resultado = Pepe.map(lambda x: (x[0],x[3],(datetime.strptime(x[1],'%H:%M') - datetime.strptime(x[2],'%H:%M'))))
ResultadoFinal3 = Resultado.map(lambda x: ((x[0], x[1]), x[2].seconds/60) )
ResultadoFinal3.reduce(lambda a,b : a if a[1]>b[1] else b )[0]

(5, '20200321')

### E) Cuál es el vuelo (numero de vuelo + fecha) con mayor retraso en el horario de llegada.

In [74]:
VuelosProgramadosRetraso = vuelosRDD.map(lambda x: (x[0], (x[7], x[6])))
VuelosProgramadosRetraso.collect()

[(1, ('07:05', '20200320')),
 (2, ('14:15', '20200320')),
 (3, ('19:30', '20200320')),
 (4, ('22:40', '20200320')),
 (5, ('08:00', '20200321')),
 (6, ('04:20', '20200322')),
 (7, ('18:20', '20200322')),
 (8, ('20:15', '20200322')),
 (9, ('14:00', '20200323')),
 (10, ('15:05', '20200323'))]

In [75]:
EstadosRetrasoFiltrado = estadosRDD.filter(lambda x: x[6] == 'Finalizado')
EstadosRetraso =  EstadosRetrasoFiltrado.map(lambda x: (x[0],x[5]))
EstadosRetraso.collect()

[(1, '07:05'), (2, '14:28'), (5, '08:30'), (7, '18:20'), (10, '15:05')]

In [76]:
Realizados = EstadosRetraso.leftOuterJoin(VuelosProgramadosRetraso)
Realizados.collect()

[(1, ('07:05', ('07:05', '20200320'))),
 (5, ('08:30', ('08:00', '20200321'))),
 (2, ('14:28', ('14:15', '20200320'))),
 (10, ('15:05', ('15:05', '20200323'))),
 (7, ('18:20', ('18:20', '20200322')))]

In [77]:
Pepe = Realizados.map(lambda x: (x[0],x[1][0] ,x[1][1][0], x[1][1][1]))
Pepe.collect()

[(1, '07:05', '07:05', '20200320'),
 (5, '08:30', '08:00', '20200321'),
 (2, '14:28', '14:15', '20200320'),
 (10, '15:05', '15:05', '20200323'),
 (7, '18:20', '18:20', '20200322')]

In [78]:
Resultado = Pepe.map(lambda x: (x[0],x[3],(datetime.strptime(x[1],'%H:%M') - datetime.strptime(x[2],'%H:%M'))))
ResultadoFinal3 = Resultado.map(lambda x: ((x[0], x[1]), x[2].seconds/60) )
ResultadoFinal3.reduce(lambda a,b : a if a[1]>b[1] else b )[0]

(5, '20200321')

### F) Cuál es la aerolínea más puntual.

In [79]:
vuelos = estadosRDD.filter(lambda x: x[6] == 'Finalizado').map(lambda x: (x[0],(x[1],x[3],x[5])))
vuelos.collect()

[(1, ('LATAM', '06:15', '07:05')),
 (2, ('AMERICAN', '12:00', '14:28')),
 (5, ('IBERIA', '06:23', '08:30')),
 (7, ('IBERIA', '15:15', '18:20')),
 (10, ('LATAM', '14:45', '15:05'))]

In [80]:
VuelosProgramadosRetraso = vuelosRDD.map(lambda x: (x[0], (x[5], x[7])))
VuelosProgramadosRetraso.collect()

[(1, ('06:15', '07:05')),
 (2, ('12:00', '14:15')),
 (3, ('17:15', '19:30')),
 (4, ('21:00', '22:40')),
 (5, ('06:00', '08:00')),
 (6, ('21:00', '04:20')),
 (7, ('15:15', '18:20')),
 (8, ('17:00', '20:15')),
 (9, ('09:10', '14:00')),
 (10, ('14:45', '15:05'))]

In [81]:
tiempo_atrasado = vuelos.join(VuelosProgramadosRetraso).map(lambda x: (x[1][0][0], datetime.strptime(x[1][0][1],'%H:%M')-datetime.strptime(x[1][1][0],'%H:%M'), datetime.strptime(x[1][0][2],'%H:%M')-datetime.strptime(x[1][1][1],'%H:%M'))).map(lambda x: (x[0],x[1].seconds/60,x[2].seconds/60))
tiempo_atrasado.collect()

[('LATAM', 0.0, 0.0),
 ('IBERIA', 23.0, 30.0),
 ('AMERICAN', 0.0, 13.0),
 ('LATAM', 0.0, 0.0),
 ('IBERIA', 0.0, 0.0)]

## 4- Se tiene un RDD con las coordenadas de rectángulos de la forma (x1,x2,y1,y2). Se pide programar en PySpark un programa que encuentre el rectángulo de superficie mínima que contiene al punto (w,z)   

In [82]:
Coordenadas = [('marca1', 'modelo1', 5, 1),
 ('marca1', 'modelo1', 10 , 1),
 ('marca2', 'modelo2', 23, 1),
 ('marca1', 'modelo1', 12, 1),
('marca2', 'modelo2', 45, 1),
('marca1', 'modelo1', 8, 1)]

In [83]:
CoordenadasRDD = sc.parallelize(Coordenadas, 8)
CoordenadasRDD.collect()

[('marca1', 'modelo1', 5, 1),
 ('marca1', 'modelo1', 10, 1),
 ('marca2', 'modelo2', 23, 1),
 ('marca1', 'modelo1', 12, 1),
 ('marca2', 'modelo2', 45, 1),
 ('marca1', 'modelo1', 8, 1)]

In [84]:
Ventas = CoordenadasRDD.map(lambda x: ((x[0], x[1]), (x[2], x[3]) ) )

In [85]:
Ventas.collect()

[(('marca1', 'modelo1'), (5, 1)),
 (('marca1', 'modelo1'), (10, 1)),
 (('marca2', 'modelo2'), (23, 1)),
 (('marca1', 'modelo1'), (12, 1)),
 (('marca2', 'modelo2'), (45, 1)),
 (('marca1', 'modelo1'), (8, 1))]

In [86]:
#Te lo debo

## 5- Se tiene un RDD con libros en donde cada registro es un texto. Se pide obtener todos los anagramas de mas de 7 letras que puedan encontrarse. El formato de salida debe ser una lista de listas en donde cada lista tiene un conjunto de palabras que son anagramas. Ejemplo: [[discounter,introduces,reductions],[percussion,supersonic]...]   

In [87]:
#Te lo debo

## 6- UBER almacena en un cluster todos los datos sobre el movimiento y viajes de todos sus vehículos. Existe un proceso que nos devuelve un RDD llamado trip_summary con los siguientes campos: (driver_id, car_id, trip_id, customer_id, date (YYYYMMDD), distance_traveled), Programar usando Py Spark un programa que nos indique cual fue el conductor con mayor promedio de distancia recorrida por viaje para Abril de 2016. 

In [88]:
trips = [
    (1, 1, 1, 1, '20160101', 10),
    (2, 2, 2, 2, '20160202', 20),
    (1, 1, 3, 1, '20160402', 15),
    (1, 1, 4, 3, '20160405', 20),
    (2, 2, 5, 4, '20160410', 25),
    (3, 3, 6, 3, '20160415', 15),
    (2, 2, 7, 1, '20160420', 40),
    (3, 3, 8, 2, '20160505', 80)
]

In [89]:
data = sc.parallelize(trips);

In [90]:
data.collect()

[(1, 1, 1, 1, '20160101', 10),
 (2, 2, 2, 2, '20160202', 20),
 (1, 1, 3, 1, '20160402', 15),
 (1, 1, 4, 3, '20160405', 20),
 (2, 2, 5, 4, '20160410', 25),
 (3, 3, 6, 3, '20160415', 15),
 (2, 2, 7, 1, '20160420', 40),
 (3, 3, 8, 2, '20160505', 80)]

In [91]:
data_filtrado = data.filter(lambda x: x[4]>'20160331' and x[4]<'20160501')
data_filtrado.collect()

[(1, 1, 3, 1, '20160402', 15),
 (1, 1, 4, 3, '20160405', 20),
 (2, 2, 5, 4, '20160410', 25),
 (3, 3, 6, 3, '20160415', 15),
 (2, 2, 7, 1, '20160420', 40)]

In [92]:
data_filtrado.map(lambda x: (x[0],(x[5],1))).reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1]))\
.collect()

[(2, (65, 2)), (1, (35, 2)), (3, (15, 1))]

In [93]:
data_filtrado.map(lambda x: (x[0],(x[5],1))).reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1]))\
.map(lambda x: (x[0],float(x[1][0]/x[1][1]))).reduce(lambda x,y: (x if x[1]>y[1] else y))

(2, 32.5)

In [94]:
data_filtrado.map(lambda x: (x[0],(x[5],1))).reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1]))\
.collect()

[(2, (65, 2)), (1, (35, 2)), (3, (15, 1))]

In [95]:
#aca me ahorro un map
data_filtrado.map(lambda x: (x[0],(x[5],1))).reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1]))\
.reduce(lambda x,y: x if float(x[1][0]/x[1][1]) > float(y[1][0]/y[1][1]) else y)

(2, (65, 2))

## 7- Una red social almacena el contenido de los chats entre sus usuarios en un RDD que tiene registros con el siguiente formato: (chat_id, user_id, nickname, text). Queremos saber cuál es el usuario (user_id) que mas preguntas hace en sus chats, contabilizamos una pregunta por cada caracter “?” que aparezca en el campo text. Programar en Spark un programa que identifique al usuario preguntón.


In [96]:
chats = [
    (1, 1, 'damu', 'Qué es esto?'),
    (2, 2, 'martin', 'Un chat!???'),
    (3, 1, 'damu', 'Ahhh! Y de donde salio? Whatsapp?'),
    (4, 2, 'martin', 'Sí! Cómo sabias????'),
    (5, 1, 'damu', 'Adivine'),
    (6, 3, 'luis', 'Hola!')
]

In [97]:
data = sc.parallelize(chats);

In [98]:
data.collect()

[(1, 1, 'damu', 'Qué es esto?'),
 (2, 2, 'martin', 'Un chat!???'),
 (3, 1, 'damu', 'Ahhh! Y de donde salio? Whatsapp?'),
 (4, 2, 'martin', 'Sí! Cómo sabias????'),
 (5, 1, 'damu', 'Adivine'),
 (6, 3, 'luis', 'Hola!')]

In [99]:
data.map(lambda x: (x[1], x[3].count('?'))).reduceByKey(lambda x,y: (x+y)).collect()

[(2, 7), (1, 3), (3, 0)]

In [100]:
data.map(lambda x: (x[1], x[3].count('?'))).reduceByKey(lambda x,y: (x+y)).reduce(lambda x,y: x if x[1]>y[1] else y)

(2, 7)

## 8- Contamos con un cluster que tiene 4 computadoras. Queremos aprovechar el paralelismo del cluster para calcular los números primos entre 2 y 20.000.000. Para esto usaremos el conocido algoritmo de la criba de Eratóstenes. Por ejemplo si empezamos con una lista de tipo (2,3,4,5,6,7,8...) en un primer paso eliminamos todos los que son mayores a 2 y divisibles por 2 y nos queda (2,3,5,7,9,11,13…) luego eliminamos todos los mayores a 3 divisibles por 3 y nos queda (2,3,5,7,11,13….etc) luego todos los divisibles por 5 y así sucesivamente. El resultado final es una lista de números que son primos. Programar este programa usando PySpark.

## 9- Se cuenta con un RDD con información sobre patentamientos de autos con la siguiente información (patente, marca, modelo, versión, tipo_vehiculo, provincia, fecha), donde tipo_vehiculo indica si la unidad patentada es auto, pickup, camión o moto. Se pide generar un programa en pySpark que indique la marca y modelo del auto más patentado por tipo de vehículo en la provincia de Buenos Aires en el mes de Abril de 2017.

In [101]:
patentamientos = [(0,
  'Marca 3',
  'Modelo 1',
  'Version 1',
  'Camion',
  'Buenos Aires',
  '2017-01-27'),
 (1, 'Marca 3', 'Modelo 3', 'Version 3', 'Pickup', 'Cordoba', '2017-01-04'),
 (2, 'Marca 3', 'Modelo 3', 'Version 2', 'Auto', 'Rio Negro', '2017-04-04'),
 (3, 'Marca 3', 'Modelo 3', 'Version 1', 'Camion', 'Santa Fe', '2017-02-04'),
 (4, 'Marca 2', 'Modelo 1', 'Version 2', 'Camion', 'Cordoba', '2017-04-28'),
 (5, 'Marca 2', 'Modelo 3', 'Version 1', 'Auto', 'Santa Cruz', '2017-03-14'),
 (6, 'Marca 1', 'Modelo 2', 'Version 2', 'Moto', 'Cordoba', '2017-02-02'),
 (7,
  'Marca 3',
  'Modelo 1',
  'Version 3',
  'Camion',
  'Buenos Aires',
  '2017-02-18'),
 (8, 'Marca 3', 'Modelo 2', 'Version 2', 'Moto', 'Buenos Aires', '2017-04-29'),
 (9, 'Marca 3', 'Modelo 2', 'Version 2', 'Camion', 'Rio Negro', '2017-04-30'),
 (10, 'Marca 1', 'Modelo 2', 'Version 1', 'Moto', 'Santa Cruz', '2017-01-18'),
 (11, 'Marca 3', 'Modelo 3', 'Version 3', 'Auto', 'Neuquen', '2017-03-16'),
 (12,
  'Marca 1',
  'Modelo 2',
  'Version 3',
  'Moto',
  'Buenos Aires',
  '2017-03-26'),
 (13, 'Marca 2', 'Modelo 2', 'Version 3', 'Auto', 'Santa Cruz', '2017-04-05'),
 (14,
  'Marca 1',
  'Modelo 3',
  'Version 1',
  'Camion',
  'Santa Cruz',
  '2017-04-27'),
 (15, 'Marca 2', 'Modelo 2', 'Version 2', 'Moto', 'Neuquen', '2017-01-28'),
 (16, 'Marca 3', 'Modelo 1', 'Version 3', 'Pickup', 'Cordoba', '2017-03-31'),
 (17, 'Marca 2', 'Modelo 2', 'Version 1', 'Auto', 'Cordoba', '2017-02-09'),
 (18, 'Marca 3', 'Modelo 3', 'Version 3', 'Auto', 'Neuquen', '2017-04-22'),
 (19, 'Marca 2', 'Modelo 3', 'Version 3', 'Auto', 'Neuquen', '2017-04-25'),
 (20,
  'Marca 2',
  'Modelo 1',
  'Version 1',
  'Camion',
  'Santa Cruz',
  '2017-02-10'),
 (21, 'Marca 3', 'Modelo 2', 'Version 3', 'Moto', 'Rio Negro', '2017-04-29'),
 (22, 'Marca 2', 'Modelo 3', 'Version 3', 'Camion', 'Cordoba', '2017-02-24'),
 (23, 'Marca 1', 'Modelo 3', 'Version 1', 'Auto', 'Cordoba', '2017-02-28'),
 (24,
  'Marca 1',
  'Modelo 3',
  'Version 3',
  'Auto',
  'Buenos Aires',
  '2017-01-03'),
 (25, 'Marca 1', 'Modelo 3', 'Version 2', 'Camion', 'Santa Fe', '2017-02-19'),
 (26, 'Marca 1', 'Modelo 1', 'Version 2', 'Moto', 'Neuquen', '2017-02-26'),
 (27, 'Marca 1', 'Modelo 3', 'Version 1', 'Camion', 'Neuquen', '2017-03-19'),
 (28, 'Marca 2', 'Modelo 1', 'Version 2', 'Camion', 'Rio Negro', '2017-04-11'),
 (29, 'Marca 1', 'Modelo 3', 'Version 1', 'Pickup', 'Cordoba', '2017-01-17'),
 (30, 'Marca 3', 'Modelo 2', 'Version 1', 'Pickup', 'Cordoba', '2017-04-22'),
 (31,
  'Marca 3',
  'Modelo 3',
  'Version 2',
  'Pickup',
  'Santa Cruz',
  '2017-04-23'),
 (32, 'Marca 1', 'Modelo 3', 'Version 3', 'Auto', 'Santa Cruz', '2017-04-17'),
 (33,
  'Marca 1',
  'Modelo 2',
  'Version 2',
  'Auto',
  'Buenos Aires',
  '2017-01-24'),
 (34, 'Marca 1', 'Modelo 1', 'Version 3', 'Camion', 'Rio Negro', '2017-01-06'),
 (35, 'Marca 2', 'Modelo 1', 'Version 3', 'Pickup', 'Cordoba', '2017-04-18'),
 (36, 'Marca 2', 'Modelo 1', 'Version 3', 'Moto', 'Rio Negro', '2017-03-02'),
 (37, 'Marca 1', 'Modelo 3', 'Version 1', 'Auto', 'Cordoba', '2017-03-30'),
 (38, 'Marca 1', 'Modelo 3', 'Version 3', 'Auto', 'Neuquen', '2017-03-16'),
 (39, 'Marca 2', 'Modelo 1', 'Version 3', 'Auto', 'Cordoba', '2017-03-20'),
 (40, 'Marca 3', 'Modelo 2', 'Version 3', 'Pickup', 'Cordoba', '2017-04-16'),
 (41,
  'Marca 2',
  'Modelo 3',
  'Version 1',
  'Moto',
  'Buenos Aires',
  '2017-01-29'),
 (42, 'Marca 2', 'Modelo 2', 'Version 3', 'Moto', 'Neuquen', '2017-02-26'),
 (43,
  'Marca 1',
  'Modelo 3',
  'Version 3',
  'Camion',
  'Santa Cruz',
  '2017-04-14'),
 (44,
  'Marca 2',
  'Modelo 3',
  'Version 3',
  'Moto',
  'Buenos Aires',
  '2017-03-29'),
 (45, 'Marca 3', 'Modelo 1', 'Version 1', 'Camion', 'Neuquen', '2017-03-07'),
 (46, 'Marca 2', 'Modelo 1', 'Version 3', 'Camion', 'Rio Negro', '2017-02-04'),
 (47, 'Marca 2', 'Modelo 2', 'Version 3', 'Pickup', 'Cordoba', '2017-02-21'),
 (48, 'Marca 2', 'Modelo 3', 'Version 1', 'Pickup', 'Cordoba', '2017-02-07'),
 (49,
  'Marca 2',
  'Modelo 2',
  'Version 1',
  'Pickup',
  'Santa Cruz',
  '2017-04-07'),
 (50, 'Marca 2', 'Modelo 2', 'Version 2', 'Moto', 'Rio Negro', '2017-01-12'),
 (51, 'Marca 1', 'Modelo 1', 'Version 2', 'Camion', 'Rio Negro', '2017-01-30'),
 (52, 'Marca 1', 'Modelo 2', 'Version 1', 'Camion', 'Rio Negro', '2017-01-23'),
 (53, 'Marca 1', 'Modelo 1', 'Version 3', 'Moto', 'Santa Fe', '2017-02-14'),
 (54, 'Marca 1', 'Modelo 3', 'Version 1', 'Auto', 'Rio Negro', '2017-03-19'),
 (55,
  'Marca 3',
  'Modelo 1',
  'Version 3',
  'Camion',
  'Buenos Aires',
  '2017-01-22'),
 (56,
  'Marca 1',
  'Modelo 2',
  'Version 1',
  'Auto',
  'Buenos Aires',
  '2017-01-23'),
 (57,
  'Marca 2',
  'Modelo 1',
  'Version 1',
  'Camion',
  'Santa Cruz',
  '2017-03-19'),
 (58, 'Marca 2', 'Modelo 2', 'Version 3', 'Moto', 'Cordoba', '2017-01-01'),
 (59, 'Marca 3', 'Modelo 2', 'Version 2', 'Auto', 'Neuquen', '2017-04-09'),
 (60, 'Marca 1', 'Modelo 1', 'Version 1', 'Moto', 'Santa Cruz', '2017-04-17'),
 (61, 'Marca 2', 'Modelo 2', 'Version 3', 'Pickup', 'Cordoba', '2017-02-15'),
 (62,
  'Marca 1',
  'Modelo 3',
  'Version 3',
  'Camion',
  'Buenos Aires',
  '2017-03-20'),
 (63, 'Marca 3', 'Modelo 1', 'Version 2', 'Moto', 'Santa Cruz', '2017-03-22'),
 (64, 'Marca 2', 'Modelo 3', 'Version 1', 'Auto', 'Santa Cruz', '2017-01-06'),
 (65,
  'Marca 1',
  'Modelo 3',
  'Version 1',
  'Pickup',
  'Santa Cruz',
  '2017-01-31'),
 (66, 'Marca 1', 'Modelo 2', 'Version 1', 'Pickup', 'Cordoba', '2017-03-01'),
 (67,
  'Marca 3',
  'Modelo 1',
  'Version 2',
  'Pickup',
  'Buenos Aires',
  '2017-04-12'),
 (68, 'Marca 1', 'Modelo 1', 'Version 2', 'Moto', 'Neuquen', '2017-01-22'),
 (69, 'Marca 1', 'Modelo 2', 'Version 3', 'Moto', 'Rio Negro', '2017-01-07'),
 (70, 'Marca 1', 'Modelo 2', 'Version 3', 'Auto', 'Santa Fe', '2017-03-05'),
 (71, 'Marca 1', 'Modelo 1', 'Version 3', 'Camion', 'Cordoba', '2017-03-03'),
 (72, 'Marca 2', 'Modelo 2', 'Version 1', 'Camion', 'Cordoba', '2017-02-20'),
 (73,
  'Marca 2',
  'Modelo 2',
  'Version 2',
  'Moto',
  'Buenos Aires',
  '2017-03-20'),
 (74, 'Marca 1', 'Modelo 1', 'Version 3', 'Moto', 'Rio Negro', '2017-01-29'),
 (75, 'Marca 2', 'Modelo 1', 'Version 2', 'Camion', 'Rio Negro', '2017-01-10'),
 (76, 'Marca 3', 'Modelo 3', 'Version 2', 'Auto', 'Cordoba', '2017-01-03'),
 (77, 'Marca 3', 'Modelo 2', 'Version 2', 'Auto', 'Neuquen', '2017-01-03'),
 (78,
  'Marca 1',
  'Modelo 1',
  'Version 2',
  'Camion',
  'Buenos Aires',
  '2017-02-18'),
 (79, 'Marca 3', 'Modelo 2', 'Version 1', 'Pickup', 'Rio Negro', '2017-02-15'),
 (80, 'Marca 2', 'Modelo 1', 'Version 1', 'Auto', 'Santa Fe', '2017-04-02'),
 (81, 'Marca 1', 'Modelo 1', 'Version 1', 'Pickup', 'Neuquen', '2017-02-12'),
 (82, 'Marca 3', 'Modelo 1', 'Version 3', 'Camion', 'Rio Negro', '2017-03-19'),
 (83, 'Marca 3', 'Modelo 2', 'Version 1', 'Camion', 'Santa Fe', '2017-01-15'),
 (84, 'Marca 3', 'Modelo 2', 'Version 1', 'Auto', 'Cordoba', '2017-03-12'),
 (85,
  'Marca 3',
  'Modelo 1',
  'Version 1',
  'Camion',
  'Buenos Aires',
  '2017-03-26'),
 (86, 'Marca 3', 'Modelo 2', 'Version 3', 'Moto', 'Santa Fe', '2017-02-19'),
 (87, 'Marca 3', 'Modelo 3', 'Version 3', 'Camion', 'Rio Negro', '2017-01-08'),
 (88, 'Marca 2', 'Modelo 2', 'Version 1', 'Auto', 'Rio Negro', '2017-04-19'),
 (89, 'Marca 3', 'Modelo 3', 'Version 1', 'Auto', 'Santa Fe', '2017-03-03'),
 (90, 'Marca 2', 'Modelo 1', 'Version 3', 'Auto', 'Cordoba', '2017-01-15'),
 (91,
  'Marca 1',
  'Modelo 1',
  'Version 2',
  'Moto',
  'Buenos Aires',
  '2017-03-30'),
 (92, 'Marca 3', 'Modelo 3', 'Version 1', 'Camion', 'Rio Negro', '2017-02-03'),
 (93, 'Marca 2', 'Modelo 2', 'Version 1', 'Pickup', 'Santa Fe', '2017-02-23'),
 (94,
  'Marca 2',
  'Modelo 2',
  'Version 1',
  'Camion',
  'Buenos Aires',
  '2017-03-29'),
 (95, 'Marca 3', 'Modelo 3', 'Version 3', 'Pickup', 'Cordoba', '2017-01-04'),
 (96,
  'Marca 1',
  'Modelo 1',
  'Version 3',
  'Pickup',
  'Buenos Aires',
  '2017-03-15'),
 (97, 'Marca 2', 'Modelo 1', 'Version 1', 'Camion', 'Santa Fe', '2017-04-28'),
 (98, 'Marca 3', 'Modelo 2', 'Version 1', 'Moto', 'Neuquen', '2017-04-03'),
 (99,
  'Marca 2',
  'Modelo 3',
  'Version 3',
  'Camion',
  'Santa Cruz',
  '2017-03-03')]

patentamientosRDD = sc.parallelize(patentamientos, 8)
patentamientosRDD.take(5)

[(0,
  'Marca 3',
  'Modelo 1',
  'Version 1',
  'Camion',
  'Buenos Aires',
  '2017-01-27'),
 (1, 'Marca 3', 'Modelo 3', 'Version 3', 'Pickup', 'Cordoba', '2017-01-04'),
 (2, 'Marca 3', 'Modelo 3', 'Version 2', 'Auto', 'Rio Negro', '2017-04-04'),
 (3, 'Marca 3', 'Modelo 3', 'Version 1', 'Camion', 'Santa Fe', '2017-02-04'),
 (4, 'Marca 2', 'Modelo 1', 'Version 2', 'Camion', 'Cordoba', '2017-04-28')]

Se pide generar un programa en pySpark que indique la marca y modelo del auto más patentado por tipo de vehículo en la provincia de Buenos Aires en el mes de Abril de 2017.

In [102]:
#Filtro por Mes y Año
#Patentamientos = Patentamientos.filter(lambda x: x[6].startswith('2017-04')
patentamiento_abril = patentamientosRDD.filter(lambda x: x[6]> '2017-03-31' and x[6]< '2017-05-01')
patentamiento_abril.take(5)

[(2, 'Marca 3', 'Modelo 3', 'Version 2', 'Auto', 'Rio Negro', '2017-04-04'),
 (4, 'Marca 2', 'Modelo 1', 'Version 2', 'Camion', 'Cordoba', '2017-04-28'),
 (8, 'Marca 3', 'Modelo 2', 'Version 2', 'Moto', 'Buenos Aires', '2017-04-29'),
 (9, 'Marca 3', 'Modelo 2', 'Version 2', 'Camion', 'Rio Negro', '2017-04-30'),
 (13, 'Marca 2', 'Modelo 2', 'Version 3', 'Auto', 'Santa Cruz', '2017-04-05')]

In [103]:
patentamientos_org = patentamiento_abril.map(lambda x: ((x[4],x[1],x[2]),1)).reduceByKey(lambda x,y: x+y)
patentamientos_org.take(5)

[(('Auto', 'Marca 2', 'Modelo 2'), 2),
 (('Pickup', 'Marca 3', 'Modelo 2'), 2),
 (('Auto', 'Marca 3', 'Modelo 3'), 2),
 (('Camion', 'Marca 1', 'Modelo 3'), 2),
 (('Auto', 'Marca 1', 'Modelo 3'), 1)]

In [104]:
patentamientos_org = patentamientos_org.map(lambda x: ( x[0][0],( (x[0][1], x[0][2]) , x[1] ) ) )
patentamientos_org.take(5)

[('Auto', (('Marca 2', 'Modelo 2'), 2)),
 ('Pickup', (('Marca 3', 'Modelo 2'), 2)),
 ('Auto', (('Marca 3', 'Modelo 3'), 2)),
 ('Camion', (('Marca 1', 'Modelo 3'), 2)),
 ('Auto', (('Marca 1', 'Modelo 3'), 1))]

In [105]:
patentamientos_org.reduceByKey(lambda a,b: a if a[1] > b[1] else b ).take(5)

[('Auto', (('Marca 3', 'Modelo 3'), 2)),
 ('Camion', (('Marca 2', 'Modelo 1'), 3)),
 ('Moto', (('Marca 3', 'Modelo 2'), 3)),
 ('Pickup', (('Marca 3', 'Modelo 2'), 2))]

## 10- A partir de la plataforma online (e-shop) de los países en los que opera, Nintendo tiene información de ventas de videojuegos diarias digitales por país en el siguiente RDD: (id_videojuego, codigo_pais, fecha, visitas_diarias, total_ventas_diarias). Por otro lado se tienen otro RDD que tiene información de todos los videojuegos que se venden en su plataforma con el siguiente formato (id_videojuego, titulo, rating_pegi, rating_esbr). Tener en cuenta que un mismo videojuego se puede vender en distintos países y esos nos permitirá obtener métricas a nivel global. 

In [106]:
InfoJuego = [(4, 'Mario Kart 8 Deluxe', 66, 21),
 (3, "Luigi's Mansion", 53, 94),
 (1, 'Kirby', 26, 84),
 (1, 'Kirby', 16, 6),
 (6, 'Super Mario Bros', 56, 19),
 (5, 'The Legend of Zelda', 97, 25),
 (2, 'Super Mario Maker 2', 51, 88),
 (6, 'Super Mario Bros', 100, 13),
 (4, 'Mario Kart 8 Deluxe', 59, 12),
 (2, 'Super Mario Maker 2', 56, 89),
 (4, 'Mario Kart 8 Deluxe', 33, 41),
 (3, "Luigi's Mansion", 24, 77),
 (7, 'Astral Chain', 27, 62),
 (4, 'Mario Kart 8 Deluxe', 84, 44),
 (4, 'Mario Kart 8 Deluxe', 74, 1),
 (5, 'The Legend of Zelda', 71, 100),
 (7, 'Astral Chain', 95, 56),
 (6, 'Super Mario Bros', 16, 84),
 (3, "Luigi's Mansion", 54, 91),
 (7, 'Astral Chain', 34, 11),
 (4, 'Mario Kart 8 Deluxe', 76, 15),
 (2, 'Super Mario Maker 2', 9, 91),
 (6, 'Super Mario Bros', 59, 1),
 (3, "Luigi's Mansion", 56, 73),
 (1, 'Kirby', 41, 13),
 (2, 'Super Mario Maker 2', 70, 33),
 (4, 'Mario Kart 8 Deluxe', 10, 73),
 (7, 'Astral Chain', 77, 32),
 (3, "Luigi's Mansion", 4, 22),
 (1, 'Kirby', 63, 36),
 (1, 'Kirby', 53, 78),
 (3, "Luigi's Mansion", 35, 63),
 (6, 'Super Mario Bros', 54, 84),
 (6, 'Super Mario Bros', 76, 94),
 (4, 'Mario Kart 8 Deluxe', 7, 30),
 (2, 'Super Mario Maker 2', 57, 26),
 (7, 'Astral Chain', 35, 30),
 (3, "Luigi's Mansion", 79, 93),
 (4, 'Mario Kart 8 Deluxe', 27, 6),
 (6, 'Super Mario Bros', 14, 3),
 (7, 'Astral Chain', 73, 57),
 (6, 'Super Mario Bros', 56, 28),
 (5, 'The Legend of Zelda', 16, 26),
 (1, 'Kirby', 56, 35),
 (2, 'Super Mario Maker 2', 70, 86),
 (1, 'Kirby', 46, 99),
 (7, 'Astral Chain', 98, 63),
 (2, 'Super Mario Maker 2', 78, 43),
 (1, 'Kirby', 88, 81),
 (6, 'Super Mario Bros', 76, 80)]

In [107]:
VentaJuegos = [
  (4, 3, '2019-08-19', 435, 148),
 (3, 1, '2019-12-12', 313, 88),
 (1, 9, '2019-09-19', 394, 322),
 (1, 9, '2019-07-31', 181, 201),
 (6, 6, '2019-10-13', 353, 194),
 (5, 8, '2019-09-21', 131, 155),
 (2, 2, '2019-03-15', 390, 9),
 (6, 8, '2019-12-11', 146, 11),
 (4, 1, '2019-08-12', 178, 151),
 (2, 1, '2019-01-22', 349, 345),
 (4, 5, '2019-05-27', 291, 274),
 (3, 3, '2019-08-05', 261, 262),
 (7, 3, '2019-04-26', 454, 82),
 (4, 1, '2019-08-24', 84, 94),
 (4, 9, '2019-06-05', 160, 84),
 (5, 4, '2019-08-16', 252, 24),
 (7, 3, '2019-09-16', 30, 149),
 (6, 10, '2019-11-16', 441, 309),
 (3, 6, '2019-11-17', 104, 260),
 (7, 3, '2019-04-30', 371, 343),
 (4, 3, '2019-04-13', 33, 29),
 (2, 6, '2019-04-11', 67, 167),
 (6, 5, '2019-08-21', 246, 308),
 (3, 5, '2019-07-23', 264, 98),
 (1, 1, '2019-01-28', 173, 337),
 (2, 3, '2019-10-20', 55, 284),
 (4, 2, '2019-10-05', 22, 10),
 (7, 9, '2019-01-02', 59, 157),
 (3, 9, '2019-07-08', 136, 299),
 (1, 9, '2019-03-25', 408, 303),
 (1, 6, '2019-04-14', 195, 15),
 (3, 10, '2019-08-26', 370, 313),
 (6, 2, '2019-04-25', 274, 203),
 (6, 2, '2019-02-06', 408, 64),
 (4, 7, '2019-11-13', 169, 209),
 (2, 4, '2019-01-07', 391, 318),
 (7, 6, '2019-08-17', 434, 271),
 (3, 10, '2019-05-25', 351, 262),
 (4, 5, '2019-01-12', 445, 168),
 (6, 10, '2019-11-11', 9, 314),
 (7, 7, '2019-11-13', 406, 316),
 (6, 6, '2019-12-04', 199, 191),
 (5, 1, '2019-04-23', 35, 16),
 (1, 10, '2019-08-22', 242, 321),
 (2, 6, '2019-07-11', 174, 350),
 (1, 7, '2019-11-03', 406, 26),
 (7, 2, '2019-06-05', 287, 282),
 (2, 2, '2019-07-29', 447, 194),
 (1, 3, '2019-09-23', 231, 145),
 (6, 7, '2019-08-22', 452, 252)]

### A) Obtener el videojuego con más ventas digitales globales (es decir en todos los países) en un RDD con el siguiente formato: (id_videojuego, titulo, total), siendo total la cantidad total de ventas digitales globales

In [108]:
ventasJuegosRDD = sc.parallelize(VentaJuegos)
infoJuegosRDD = sc.parallelize(InfoJuego)

##### (id_videojuego, codigo_pais, fecha, visitas_diarias, total_ventas_diarias) 

##### (id_videojuego, titulo, rating_pegi, rating_esbr).



In [109]:
ventasJuegosRDD.take(5)

[(4, 3, '2019-08-19', 435, 148),
 (3, 1, '2019-12-12', 313, 88),
 (1, 9, '2019-09-19', 394, 322),
 (1, 9, '2019-07-31', 181, 201),
 (6, 6, '2019-10-13', 353, 194)]

In [110]:
infoJuegosRDD.take(5)

[(4, 'Mario Kart 8 Deluxe', 66, 21),
 (3, "Luigi's Mansion", 53, 94),
 (1, 'Kirby', 26, 84),
 (1, 'Kirby', 16, 6),
 (6, 'Super Mario Bros', 56, 19)]

In [111]:
ventas_por_juego = ventasJuegosRDD.map(lambda x: (x[0],x[4])).reduceByKey(lambda x,y: (x+y))
ventas_por_juego.take(5)

[(4, 1167), (6, 1846), (2, 1667), (3, 1582), (1, 1670)]

In [112]:
info_reducida = infoJuegosRDD.map(lambda x: (x[0],x[1])).reduceByKey(lambda x,y: x)
info_reducida.take(5)

[(4, 'Mario Kart 8 Deluxe'),
 (6, 'Super Mario Bros'),
 (2, 'Super Mario Maker 2'),
 (3, "Luigi's Mansion"),
 (1, 'Kirby')]

In [113]:
juego_mas_vendido = ventas_por_juego.join(info_reducida).map(lambda x: (x[0],x[1][1],x[1][0])).reduce(lambda x,y: x if x[2]>y[2] else y)
juego_mas_vendido

(6, 'Super Mario Bros', 1846)

### B) Para el videojuego con mas ventas, obtener cual es el país para el cual ser registra una mayor tasa de conversión (es decir, mayor total_ventas_diarias / visitas_diarias) 


In [114]:
ID_mas_vendido = juego_mas_vendido[0]
ID_mas_vendido

6

In [115]:
ventasJuegosRDD.take(5)

[(4, 3, '2019-08-19', 435, 148),
 (3, 1, '2019-12-12', 313, 88),
 (1, 9, '2019-09-19', 394, 322),
 (1, 9, '2019-07-31', 181, 201),
 (6, 6, '2019-10-13', 353, 194)]

In [118]:
ventasJuegosRDD.filter(lambda x: x[0] == ID_mas_vendido).map(lambda x: (x[1],(x[4],x[3]))).reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])).reduce(lambda x,y: x if x[1][0]/x[1][1] > y[1][0]/y[1][1] else y)

(10, (623, 450))